In [772]:
library(tidyverse)
library(textclean)
library(readxl)

#working_dir <- getwd()
#setwd("../")
source("base/it-402-dc-data_initial_exploration-base.r")
#setwd(working_dir)

#rm(working_dir)

In [2]:
#rm(list = ls())

In [3]:
dbConn <- dbConnect(RSQLite::SQLite(), "sta_it_402/data/sqa-data-db.sqlite")
dbConn

<SQLiteConnection>
  Path: /Users/aba-sah/OneDrive/work/programming/devt/r/work/sta/it-402-dc/sta_it_402/data/sqa-data-db.sqlite
  Extensions: TRUE

In [ ]:
dbGetQuery(dbConn,
           "SELECT * FROM sqlite_master WHERE type = 'table' AND (name = 'centre_postcodes')")

In [ ]:

## https://www.gov.scot/publications/school-level-summary-statistics

## https://www.gov.scot/publications/scottish-government-urban-rural-classification-2016/pages/2
## https://www.webarchive.org.uk/wayback/archive/20190502100317/https://www2.gov.scot/Topics/Statistics/Browse/School-Education/RollsByStage

## https://www.nrscotland.gov.uk/statistics-and-data/geography/our-products/scottish-postcode-directory/2022-1
## https://www.nrscotland.gov.uk/files/geography/Products/postcode-bkgrd-info.pdf
## https://statistics.gov.scot/slice?dataset=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fattainment-for-all&http%3A%2F%2Fpurl.org%2Flinked-data%2Fcube%23measureType=http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fmeasure-properties%2Faverage-total-tariff-score&http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fdimension%2FattainmentDistribution=http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fconcept%2Fattainment-distribution%2Fhighest-20&http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fdimension%2Fcomparator=http%3A%2F%2Fstatistics.gov.scot%2Fdef%2Fconcept%2Fcomparator%2Freal-establishment
## https://www.opendata.nhs.scot/sl/dataset/geography-codes-and-labels/resource/967937c4-8d67-4f39-974f-fd58c4acfda53fold


### school results from Education Scotland, demographic data from gov.scot and college data reqest from SFC

In [4]:
path_to_demographic_data <- paste(path_to_file_store, "demographic_data", sep = "/")
path_to_demographic_data          

[1] "sta_it_402/data/demographic_data"

In [5]:
datafiles_secondary_school_level_summary_statistics <-
    c("2016" = paste(path_to_demographic_data, "School_Contact_Details_2018.xlsx", sep = "/"), # Open at September 2016
      "2017" = paste(path_to_demographic_data, "School_Contact_Details_2018.xlsx", sep = "/"), # Open at September 2017
      "2018" = paste(path_to_demographic_data, "School_Contact_Details_2018.xlsx", sep = "/"), # Open at September 2018
      "2019" = paste(path_to_demographic_data, "School_level_summary_statistics_2019.xlsx", sep = "/"), # collected Sep 2019
      "2020" = paste(path_to_demographic_data, "School_level_summary_statistics_2020.xlsx", sep = "/"), # 2020
      "2021" = paste(path_to_demographic_data, "School_level_summary_statistics_2021.xlsx", sep = "/"), # 2021 School Level Statistics
      "2022" = paste(path_to_demographic_data, "School_level_summary_statistics_2022.xlsx", sep = "/") #  sep = "/") # 2022 School Level Statistics
      )
#datafiles_secondary_school_level_summary_statistics <- 
#    datafiles_secondary_school_level_summary_statistics[!str_detect(datafiles_secondary_school_level_summary_statistics, "\\~")] # just in casea files open on sname machine while reading dir

datafiles_secondary_school_level_summary_statistics_sheetname <-
    c("2016" = "Open at September 2016",
      "2017" = "Open at September 2017",
      "2018" = "Open at September 2018",
      "2019" = "2019",
      "2020" = "2020",
      "2021" = "2021 School Level Statistics",
      "2022" = "2022 School Level Statistics"
      )


datafiles_cs_entries <- list.files("fiona_projects/data/", pattern = "stat00007", full.names = TRUE, ignore.case = TRUE, recursive = TRUE)
datafiles_cs_entries <- datafiles_cs_entries[!str_detect(datafiles_cs_entries, "\\~")] 

sel_idx <- str_detect(datafiles_cs_entries, ".csv")
datafiles_cs_entries[sel_idx]

sel_idx2 <- str_detect(datafiles_cs_entries, "dictionary|csv")
datafiles_cs_entries[!sel_idx2]

[1] "fiona_projects/data//stat00007_graded_v4.csv"  
[2] "fiona_projects/data//stat00007_ungraded_v4.csv"
[3] "fiona_projects/data//stat00007_unit_v4.csv"

[1] "fiona_projects/data//stat00007.xlsx"

In [6]:
rm(path_to_demographic_data)

In [29]:
centre_postcodes <- bind_rows(lapply(datafiles_cs_entries[sel_idx], read_csv, 
                                     col_select = matches("Centre|c_type|address|postcode|ea_name|geo_area"), 
                                     name_repair = str_to_title,
                                     trim_ws = TRUE
                                    )
                             ) %>%
    rename_with(snakecase::to_upper_camel_case) %>% 

    bind_rows(read_xlsx(datafiles_cs_entries[!sel_idx2], 
                            trim_ws = TRUE, 
                       ) %>% 
              select(matches("Centre|c_type|address|postcode|ea_name|geo_area")) %>%
              rename_with(snakecase::to_upper_camel_case)
             ) %>% 
    rename(EducationAuthority = snakecase::to_upper_camel_case("ea_name")) %>%
              

    bind_rows(read_csv("fiona_projects/data/9414.csv", 
                       col_select = matches("Centre|c_type|address|postcode|(Education Authority)"), 
                       trim_ws = TRUE, 
                       skip = 1) %>%
              rename_with(snakecase::to_upper_camel_case)
             ) %>% 

    filter(rowSums(is.na(.)) != ncol(.)) %>%
    rename(CentreType = CType) %>%
    separate("CentreType", c("CentreStatus", "CentreType"), sep = "-", fill = "left") %>%

    mutate(across(where(is.character), str_squish),
           across(CentreName, ~ gsub("(^\\$)?.+$", "\\1", .), .names = "CentreClosed"),
           across(CentreName, ~ gsub("(^\\$)", "", .)),
           across(c(CentreName, matches("Address[3-9]")), ~ if_else((str_to_upper(.) == .), str_to_lower(.), .)),
           across(c(CentreName, matches("Address[2-9]")), tools::toTitleCase),
                           
           across(CentreType, ~ gsub("\\bFE\\b", "Further Education", .)),
           
           across(c(GeoArea, EducationAuthority), ~ gsub("(\\w+), City of", "\\1 City", .)), 
           across(GeoArea, ~ gsub("(Shetland|Orkney).*", "\\1 Islands", .)), 
           across(GeoArea, ~ if_else(str_detect(., "Edinburgh"), "City of Edinburgh", .)), 
           across(GeoArea, ~ if_else(str_detect(., "Clackmannan"), "Clackmannanshire", .)), 
           across(GeoArea, ~ if_else(str_detect(., "Western Isles|Eileanan|Eilean"), "Na h-Eileanan Siar", .)), 
           across(GeoArea, ~ if_else(str_detect(., regex("Grant aided", ignore_case = TRUE)), 
                                     gsub("(^|[[:space:]])([[:alpha:]])", "\\1\\U\\2", ., perl = TRUE), .)), 
           across(GeoArea, ~ str_squish(str_remove(., "Council"))),
           across(matches("GeoArea|Authority"), ~ gsub("\\band\\b", "\\&", .)),
           across(GeoArea, ~ fct_relevel(., "Grant Aided", after = Inf)),
           
           across(EducationAuthority, ~ gsub("^(.*Edinburgh.*)(\\sCouncil Education Department)$", 
                                             "The City of Edinburgh\\2", .)), 
           across(EducationAuthority, ~ gsub("^(.*Eileanan.*)(\\sCouncil Education Department)$", 
                                             "Western Isles\\2", .)), 
           
           across(Address1, ~ gsub("(^\\$)?.+$", "\\1", .), .names = "AddressFlag"),
           across(Address1, ~ gsub("(^\\$)", "", .)),
           across(c(CentreClosed, AddressFlag), ~ !is.na(.) & (str_trim(.) != "")),

           across(!matches("Name|Address") & !where(is.logical), as.factor)
          ) %>%

    arrange(GeoArea) %>% # NAs both fields, can't use reorder
    mutate(across(EducationAuthority, ~ fct_inorder(.)), # fct_reorder(., as.integer(GeoArea))),
           across(EducationAuthority, ~ fct_relevel(., "Independent", after = Inf)),
          ) %>%


    unite("Address", matches("Address\\d"), na.rm = TRUE, sep = ",\n") %>%
    relocate(CentreClosed, .after = CentreName) %>%
    relocate(AddressFlag, .after = Postcode) %>%
    
    distinct() %>%
    arrange(Centre) %>%
    glimpse()

levels(centre_postcodes$EducationAuthority)
levels(centre_postcodes$GeoArea)

levels(centre_postcodes$CentreType)
levels(centre_postcodes$CentreStatus)

length(levels(centre_postcodes$EducationAuthority))
length(levels(centre_postcodes$GeoArea))

Rows: 7561 Columns: 10
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (9): Centre_name, C_type, Address1, Address2, Address3, Address4, Postco...
dbl (1): Centre

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 11443 Columns: 10
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (9): Centre_name, C_type, Address1, Address2, Address3, Address4, Postco...
dbl (1): Centre

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 95659 Columns: 10
── Column specification ──────────────────────────────────────────────────────────
Delimiter: ","
chr (9): Centre_name, C_type, Address1, Address2, Address3, Address4, Postco...
dbl (1): Centre

ℹ Use `spec()` to retrieve the full 

Rows: 1,351
Columns: 10
$ Centre             <fct> 1000144, 1001930, 1002732, 1002732, 1002759, 100285…
$ CentreName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <fct> Education Authority, Education Authority, Education…
$ CentreType         <fct> Special School, Special School, Secondary School, N…
$ Address            <chr> "Firpark Street,\nMotherwell", "67 Townhead Road,\n…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, G67 2UF, KA13 6DE, DD3 8…
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


[1] "Aberdeen City Council Education Department"        
 [2] "Aberdeenshire Council Education Department"        
 [3] "Angus Council Education Department"                
 [4] "Argyll & Bute Council Education Department"        
 [5] "The City of Edinburgh Council Education Department"
 [6] "Clackmannanshire Council Education Department"     
 [7] "Dumfries & Galloway Council Education Department"  
 [8] "Dundee City Council Education Department"          
 [9] "East Ayrshire Council Education Department"        
[10] "East Dunbartonshire Council Education Department"  
[11] "East Lothian Council Education Department"         
[12] "East Renfrewshire Council Education Department"    
[13] "Falkirk Council Education Department"              
[14] "Fife Council Education Department"                 
[15] "City of Glasgow Council Education Department"      
[16] "Highland Council Education Department"             
[17] "Inverclyde Council Education Department"           
[18] "Midlothian Council Education Department"           
[19] "The Moray Council Education Department"            
[20] "Western Isles Council Education Department"        
[21] "North Ayrshire Council Education Department"       
[22] "North Lanarkshire Council Education Department"    
[23] "Orkney Islands Council Education Department"       
[24] "Perth & Kinross Council Education Department"      
[25] "Renfrewshire Council Education Department"         
[26] "Scottish Borders Council Education Department"     
[27] "Shetland Islands Council Education Department"     
[28] "South Ayrshire Council Education Department"       
[29] "South Lanarkshire Council Education Department"    
[30] "Stirling Council Education Department"             
[31] "West Dunbartonshire Council Education Department"  
[32] "West Lothian Council Education Department"         
[33] "Independent"

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "England"             "Falkirk"             "Far East"           
[16] "Fife"                "Glasgow City"        "Highland"           
[19] "Inverclyde"          "Middle East"         "Midlothian"         
[22] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[25] "North Lanarkshire"   "Orkney Islands"      "Other"              
[28] "Perth & Kinross"     "Renfrewshire"        "Scottish Borders"   
[31] "Shetland Islands"    "South Asia"          "South Ayrshire"     
[34] "South Lanarkshire"   "Stirling"            "Wales"              
[37] "West Dunbartonshire" "West Lothian"

[1] "Further Education Centre"         "Further Education College"       
 [3] "Higher Education Institutions"    "HM Prisons"                      
 [5] "Private Sector Employer"          "Private Sector Training Provider"
 [7] "Public Sector Employer"           "Public Sector Training Provider" 
 [9] "Secondary School"                 "Special School"                  
[11] "Voluntary Sector Organisation"

[1] "Education Authority" "Independent"

[1] 33

[1] 38

In [30]:
centre_postcodes %>%
    count(CentreClosed, AddressFlag)

centre_postcodes %>%
    filter(CentreClosed) 

centre_postcodes %>%
    filter(str_detect(CentreName, "SRUC")) 


CentreClosed,AddressFlag,n
<lgl>,<lgl>,<int>
FALSE,FALSE,1221
FALSE,TRUE,127
TRUE,FALSE,3


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
3020177,Surrey 100 Limited T/a the Normandy Hotel,TRUE,NA,Private Sector Training Provider,"The Normandy Hotel, Inchinnan Road, Renfrew",PA4 9EJ,FALSE,NA,Renfrewshire
5243831,Kincorth Academy,TRUE,Education Authority,Secondary School,"Kincorth Circle, Aberdeen",AB12 5NL,FALSE,Aberdeen City Council Education Department,Aberdeen City
5243831,Kincorth Academy,TRUE,NA,NA,,AB12 5NL,FALSE,Aberdeen City Council Education Department,NA


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
1003259,SRUC Scotland's Rural College,FALSE,NA,Further Education College,"SRUC - Oatridge Campus, Ecclesmachen, Broxburn, West Lothian",EH52 6NH,FALSE,NA,West Lothian
5460255,SRUC Elmwood Campus,FALSE,NA,Further Education College,"Carslogie Road, Cupar, Fife",KY15 4JB,FALSE,NA,Fife
5560055,SRUC Oatridge Campus,FALSE,NA,Further Education College,"Ecclesmachan, Broxburn, West Lothian",EH52 6NH,FALSE,NA,West Lothian
5960150,SRUC Barony Campus,FALSE,NA,Further Education College,"Parkgate, Dumfries, Dumfries & Galloway",DG1 3NE,FALSE,NA,Dumfries & Galloway
8274029,SRUC Ayr Campus,FALSE,NA,Further Education College,"SAC Riverside Campus, University Avenue, Ayr, Ayrshire",KA8 0SX,FALSE,NA,South Ayrshire


In [31]:
centre_postcodes %>%

    count(CentreStatus, EducationAuthority) %>% #, CentreType
    print(n = Inf)

# A tibble: 68 × 3
   CentreStatus        EducationAuthority                                     n
   <fct>               <fct>                                              <int>
 1 Education Authority Aberdeen City Council Education Department            16
 2 Education Authority Aberdeenshire Council Education Department            22
 3 Education Authority Angus Council Education Department                     8
 4 Education Authority Argyll & Bute Council Education Department             9
 5 Education Authority The City of Edinburgh Council Education Department    29
 6 Education Authority Clackmannanshire Council Education Department          4
 7 Education Authority Dumfries & Galloway Council Education Department      17
 8 Education Authority Dundee City Council Education Department              11
 9 Education Authority East Ayrshire Council Education Department            13
10 Education Authority East Dunbartonshire Council Education Department      11
11 Education Authorit

In [32]:
centre_postcodes %>%
    group_by(across(matches("Centre"))) %>%
    add_tally() %>%
    filter(n > 1)


# manually correct addresses

centre_postcodes %>%

    mutate(across(Postcode, as.character),
           across(Address, ~ case_when((Centre == "5251834") ~ "Raemoir Road,\nBanchory",
                                       (Centre == "8337934") ~ "Bellfield Road,\nKirkintilloch,\nGlasgow",
                                       #(Centre == "") ~ "",
                                       TRUE ~ .
                                      )),
           across(Postcode, ~ case_when((Centre == "5251834") ~ "AB31 5UJ",
                                        TRUE ~ .
                                       )),   
           
           across(Postcode, as.factor),           
          ) %>%
    group_by(across(matches("Centre"))) %>%
    add_tally() %>%
    filter(n > 1) %>%
    distinct()

Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,n
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>,<int>
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Raemoir Road, Banchory",AB31 5UJ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,2
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Schoolhill, Banchory",AB31 5TQ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch, Glasgow",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,2


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,n
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>,<int>
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Raemoir Road, Banchory",AB31 5UJ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch, Glasgow",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,2


In [33]:
dim(centre_postcodes)

centre_postcodes %>%
    filter(Centre == "1002732")

centre_postcodes %>%
    group_by(Centre) %>%
    add_tally() %>%
    filter(n > 1) %>%
    filter(if_all(c(CentreStatus, CentreType, GeoArea), ~ !is.na(.))) %>%
    add_tally() %>%
    filter(nn > 1)

centre_postcodes %>%
    filter(if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%
    nrow()

centre_postcodes %>%
    filter(!if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%
    nrow()

centre_postcodes %>%
    filter(if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ !is.na(.) & (. != ""))) %>%
    nrow()

centre_postcodes %>%
    filter(if_all(c(CentreType, GeoArea, Address), ~ !is.na(.) & (. != ""))) %>%
    nrow()

[1] 1351   10

Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
1002732,Cumbernauld Academy,FALSE,Education Authority,Secondary School,"Kildrum Ring Road, Kildrum, Cumbernauld",G67 2UF,FALSE,North Lanarkshire Council Education Department,North Lanarkshire
1002732,Cumbernauld Academy,FALSE,NA,NA,,G67 2UF,FALSE,North Lanarkshire Council Education Department,NA


Storing counts in `nn`, as `n` already present in input
ℹ Use `name = "new_name"` to pick a new name.


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,n,nn
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>,<int>,<int>
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Raemoir Road, Banchory",AB31 5UJ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,3,2
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Schoolhill, Banchory",AB31 5TQ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,3,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch, Glasgow",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,3,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,3,2


[1] 362

[1] 989

[1] 498

[1] 989

In [34]:
centre_postcodes %>%
    filter(!if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%

    right_join(centre_postcodes %>%
                   filter(if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%
                   select(- c(CentreStatus, CentreType, GeoArea, Address)) %>%
                   rename(EducationAuthority_fill = EducationAuthority)
    ) %>%

    mutate(across(EducationAuthority, ~ coalesce(., EducationAuthority_fill))) %>%
    filter(is.na(EducationAuthority))

Joining with `by = join_by(Centre, CentreName, CentreClosed, Postcode,
AddressFlag)`


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,EducationAuthority_fill
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>,<fct>


In [35]:
centre_postcodes %>%
    filter(!if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%

    right_join(centre_postcodes %>%
                   filter(if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%
                   select(- c(CentreStatus, CentreType, GeoArea, Address)) %>%
                   rename(EducationAuthority_fill = EducationAuthority)
    ) %>%
    #filter(if_any(c(EducationAuthority, CentreStatus), ~ (. == "Independent"))) %>%
    mutate(across(EducationAuthority, ~ case_when(if_any(c(EducationAuthority, CentreStatus), ~ (. == "Independent")) ~ .,
                                                  TRUE ~ coalesce(., EducationAuthority_fill)
                                                 ))
           ) %>%
    filter(is.na(EducationAuthority) | (CentreStatus == "Independent"))

Joining with `by = join_by(Centre, CentreName, CentreClosed, Postcode,
AddressFlag)`


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,EducationAuthority_fill
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>,<fct>
1003593,Al Khalil College,FALSE,Independent,Secondary School,"129-131 Abbotsford Place, Glasgow",G5 9SS,FALSE,NA,Glasgow City,Independent
1004891,Moore House Academy Butterstone,FALSE,Independent,Special School,"Butterstone, Dunkeld",PH8 0HJ,FALSE,NA,Perth & Kinross,Independent
5281334,Lathallan School,FALSE,Independent,Secondary School,"Brotherton Castle, Johnshaven, Montrose",DD10 0HN,FALSE,NA,Angus,Independent
5281938,Albyn School,FALSE,Independent,Secondary School,"17-23 Queen's Road, Aberdeen",AB15 4PB,FALSE,NA,Aberdeen City,Independent
5282039,Robert Gordon's College,FALSE,Independent,Secondary School,"Schoolhill, Aberdeen",AB10 1FE,FALSE,NA,Aberdeen City,Independent
5282136,St Margaret's School for Girls,FALSE,Independent,Secondary School,"17 Albyn Place, Aberdeen",AB10 1RU,FALSE,NA,Aberdeen City,Independent
5380332,Kilgraston School,FALSE,Independent,Secondary School,"Bridge of Earn, Perth",PH2 9BQ,FALSE,NA,Perth & Kinross,Independent
5380936,Strathallan School,FALSE,Independent,Secondary School,"Forgandenny, Perth",PH2 9EG,FALSE,NA,Perth & Kinross,Independent
5381436,High School of Dundee,FALSE,Independent,Secondary School,"Euclid Crescent, Dundee",DD1 1HU,FALSE,NA,Dundee City,Independent


In [36]:
centre_postcodes <- centre_postcodes %>%
    filter(!if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%

    full_join(centre_postcodes %>%
                   filter(if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%
                   select(- c(CentreStatus, CentreType, GeoArea, Address)) %>%
                   rename(EducationAuthority_fill = EducationAuthority)
    ) %>%
    mutate(across(EducationAuthority, ~ case_when(if_any(c(EducationAuthority, CentreStatus), ~ (. == "Independent")) ~ .,
                                                  TRUE ~ coalesce(., EducationAuthority_fill)
                                                 ))
           ) %>%
    select(- EducationAuthority_fill) %>%
    glimpse()

Joining with `by = join_by(Centre, CentreName, CentreClosed, Postcode,
AddressFlag)`


Rows: 995
Columns: 10
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <fct> Education Authority, Education Authority, Education…
$ CentreType         <fct> Special School, Special School, Secondary School, F…
$ Address            <chr> "Firpark Street,\nMotherwell", "67 Townhead Road,\n…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


In [37]:
levels(centre_postcodes$CentreStatus)
levels(centre_postcodes$CentreType)

centre_postcodes %>%
    filter(is.na(EducationAuthority)) %>%
    count(CentreStatus, CentreType)

centre_postcodes %>%
    filter(!is.na(CentreStatus)) %>%
    count(CentreStatus, CentreType)

centre_postcodes %>%
    filter((CentreStatus == "Education Authority") & is.na(EducationAuthority))

centre_postcodes %>%
    filter((CentreStatus == "Independent") & !is.na(EducationAuthority))

[1] "Education Authority" "Independent"

[1] "Further Education Centre"         "Further Education College"       
 [3] "Higher Education Institutions"    "HM Prisons"                      
 [5] "Private Sector Employer"          "Private Sector Training Provider"
 [7] "Public Sector Employer"           "Public Sector Training Provider" 
 [9] "Secondary School"                 "Special School"                  
[11] "Voluntary Sector Organisation"

CentreStatus,CentreType,n
<fct>,<fct>,<int>
Education Authority,Secondary School,1
Independent,Secondary School,39
Independent,Special School,27
NA,Further Education College,59
NA,Higher Education Institutions,58
NA,HM Prisons,1
NA,Private Sector Employer,37
NA,Private Sector Training Provider,225
NA,Public Sector Employer,41


CentreStatus,CentreType,n
<fct>,<fct>,<int>
Education Authority,Further Education Centre,1
Education Authority,Secondary School,354
Education Authority,Special School,77
Independent,Secondary School,39
Independent,Special School,27


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
9994971,SQA Operations,FALSE,Education Authority,Secondary School,"The Optima Building, 58 Robertson Street, Scottish Qualifications Authority, Glasgow",G2 8DQ,FALSE,NA,Glasgow City


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>


In [38]:
centre_postcodes %>%
    count(CentreStatus, CentreType)


centre_postcodes %>%
    filter(if_all(c(CentreStatus, CentreType), is.na))

CentreStatus,CentreType,n
<fct>,<fct>,<int>
Education Authority,Further Education Centre,1
Education Authority,Secondary School,354
Education Authority,Special School,77
Independent,Secondary School,39
Independent,Special School,27
NA,Further Education College,59
NA,Higher Education Institutions,58
NA,HM Prisons,1
NA,Private Sector Employer,37


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
5244838,Torry Academy,FALSE,NA,NA,NA,AB11 8HD,FALSE,Aberdeen City Council Education Department,NA
5370035,The New School Butterstone,FALSE,NA,NA,NA,PH2 9BT,FALSE,Independent,NA
5780136,Beaconhurst School,FALSE,NA,NA,NA,FK9 4RR,FALSE,Independent,NA
5939534,North West Community Campus,FALSE,NA,NA,NA,DG2 9DY,FALSE,Dumfries & Galloway Council Education Department,NA
8243131,Auchinleck Academy,FALSE,NA,NA,NA,KA18 2LY,FALSE,East Ayrshire Council Education Department,NA
8243239,Cumnock Academy,FALSE,NA,NA,NA,KA18 1EH,FALSE,East Ayrshire Council Education Department,NA


In [39]:

centre_postcodes %>%
    filter(if_all(c(CentreStatus, CentreType), is.na)) %>%
    distinct(Centre) %>%

    left_join(centre_postcodes
    )

Joining with `by = join_by(Centre)`


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
5244838,Torry Academy,FALSE,Education Authority,Secondary School,"Tullos Circle, Torry, Aberdeen",AB11 8HD,TRUE,Aberdeen City Council Education Department,Aberdeen City
5244838,Torry Academy,FALSE,NA,NA,NA,AB11 8HD,FALSE,Aberdeen City Council Education Department,NA
5370035,The New School Butterstone,FALSE,Independent,Special School,"c/o Forteviot Primary School, Perth",PH2 9BT,TRUE,NA,Perth & Kinross
5370035,The New School Butterstone,FALSE,NA,NA,NA,PH2 9BT,FALSE,Independent,NA
5780136,Beaconhurst School,FALSE,Independent,Secondary School,"52 Kenilworth Road, Bridge of Allan",FK9 4RR,TRUE,NA,Stirling
5780136,Beaconhurst School,FALSE,NA,NA,NA,FK9 4RR,FALSE,Independent,NA
5939534,North West Community Campus,FALSE,Education Authority,Secondary School,"Alloway Road, Dumfries",DG2 9DY,TRUE,Dumfries & Galloway Council Education Department,Dumfries & Galloway
5939534,North West Community Campus,FALSE,NA,NA,NA,DG2 9DY,FALSE,Dumfries & Galloway Council Education Department,NA
8243131,Auchinleck Academy,FALSE,Education Authority,Secondary School,"Sorn Road, Auchinleck, Cumnock",KA18 2LY,TRUE,East Ayrshire Council Education Department,East Ayrshire


In [40]:
# AddressFlag - keep flag and fill all else
dim(centre_postcodes)

centre_postcodes <- centre_postcodes %>%
    filter(!if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%

    full_join(centre_postcodes %>%
                    filter(if_all(c(CentreStatus, CentreType), is.na)) %>%
                    distinct(Centre) %>%

                    left_join(centre_postcodes) %>%
               
                   filter(if_all(c(CentreStatus, CentreType, GeoArea, Address), ~ is.na(.) | (. == ""))) %>%
                   select(- c(CentreStatus, CentreType, GeoArea, Address, AddressFlag)) %>% 
                   rename(EducationAuthority_fill = EducationAuthority)
    ) %>%
    mutate(across(EducationAuthority, ~ case_when(if_any(c(EducationAuthority, CentreStatus), ~ (. == "Independent")) ~ .,
                                                  TRUE ~ coalesce(., EducationAuthority_fill)
                                                 ))
           ) %>%
    select(- EducationAuthority_fill) %>%
    glimpse()

[1] 995  10

Joining with `by = join_by(Centre)`
Joining with `by = join_by(Centre, CentreName, CentreClosed, Postcode)`


Rows: 989
Columns: 10
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <fct> Education Authority, Education Authority, Education…
$ CentreType         <fct> Special School, Special School, Secondary School, F…
$ Address            <chr> "Firpark Street,\nMotherwell", "67 Townhead Road,\n…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


In [41]:
centre_postcodes %>%

    group_by(across(matches("Centre"))) %>%
    add_tally() %>%
    filter(n > 1) %>%
    arrange(Centre)

centre_postcodes %>%
    filter(is.na(CentreType)) %>%
    nrow()

centre_postcodes %>%
    filter(if_all(c(EducationAuthority, GeoArea), is.na)) %>%
    nrow()

centre_postcodes %>%
    filter(is.na(GeoArea)) %>%
    nrow()

centre_postcodes %>%
    filter(is.na(EducationAuthority)) %>%
    nrow()


centre_postcodes %>%
    count(CentreStatus, CentreType)

Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,n
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>,<int>
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Raemoir Road, Banchory",AB31 5UJ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,2
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Schoolhill, Banchory",AB31 5TQ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch, Glasgow",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,2
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire,2


[1] 0

[1] 0

[1] 0

[1] 558

CentreStatus,CentreType,n
<fct>,<fct>,<int>
Education Authority,Further Education Centre,1
Education Authority,Secondary School,354
Education Authority,Special School,77
Independent,Secondary School,39
Independent,Special School,27
NA,Further Education College,59
NA,Higher Education Institutions,58
NA,HM Prisons,1
NA,Private Sector Employer,37


In [42]:
# 5251834 - Banchory Academy - school website and google maps both have  AB31 5UJ 
# manually deleting other

centre_postcodes %>%
    distinct(Centre, Postcode) %>%

    group_by(across(matches("Centre"))) %>%
    add_tally() %>%
    filter(n > 1)

Centre,Postcode,n
<fct>,<fct>,<int>
5251834,AB31 5UJ,2
5251834,AB31 5TQ,2


In [43]:
# manual corrections

dim(centre_postcodes)

centre_postcodes <- centre_postcodes %>%
    filter(!((Centre == "5251834") & (Postcode == "AB31 5TQ"))) %>%

    mutate(across(Address, ~ case_when(#(Centre == "5251834") ~ "Raemoir Road, Banchory", - sorted with filter above
                                       (Centre == "8337934") ~ "Bellfield Road,\nKirkintilloch,\nGlasgow",
                                       #(Centre == "") ~ "",
                                       TRUE ~ .
                                      )),
          ) %>%
    distinct() %>%
    glimpse()

centre_postcodes %>%
    filter(Centre %in% c("8337934", "5251834"))

[1] 989  10

Rows: 987
Columns: 10
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <fct> Education Authority, Education Authority, Education…
$ CentreType         <fct> Special School, Special School, Secondary School, F…
$ Address            <chr> "Firpark Street,\nMotherwell", "67 Townhead Road,\n…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
5251834,Banchory Academy,FALSE,Education Authority,Secondary School,"Raemoir Road, Banchory",AB31 5UJ,FALSE,Aberdeenshire Council Education Department,Aberdeenshire
8337934,St Ninian's High School (Kirkintilloch),FALSE,Education Authority,Secondary School,"Bellfield Road, Kirkintilloch, Glasgow",G66 1DT,FALSE,East Dunbartonshire Council Education Department,East Dunbartonshire


In [44]:
dim(centre_postcodes)
n_distinct(centre_postcodes$Centre)

[1] 987  10

[1] 987

In [45]:
centre_postcodes %>%
    count(GeoArea, sort = TRUE)

GeoArea,n
<fct>,<int>
Glasgow City,117
City of Edinburgh,75
North Lanarkshire,56
Far East,54
Fife,51
Highland,50
South Lanarkshire,42
Aberdeen City,37
Renfrewshire,37


### Merge with data from school and college demographics

* ... and manually fill in remaining blanks in postcodes and local authorities

In [46]:
centre_postcodes %>%
    filter(if_all(GeoArea, ~ is.na(.) | (. == "")))

centre_postcodes %>%
    filter(if_all(Postcode, ~ is.na(.) | (. == "")))

Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
3002349,Centre of Excellence for Technical and Vocational Education,FALSE,NA,Higher Education Institutions,"Sh. Khalifa bin Salman Institute, Ministry of Education, Muharraq, PO Box 22909",NA,TRUE,NA,Middle East
3003647,ADNOC SQA Centre,FALSE,NA,Public Sector Employer,"Ms. Reem A Mohamed, SQA CENTRE, PEOPLE DEVELOPMENT FUNCTIONS, 24th Floor ADNOC HQ Corniche Street, PO Box 898 Abu Dhabi",NA,FALSE,NA,Middle East
3008576,Business and Technology Education Centre (BTEC),FALSE,NA,Private Sector Training Provider,"PO Box 43358, Room 1504, 15th Floor, Mai Tower, Al Nahda 1, Dubai",NA,TRUE,NA,Middle East
3017915,Racer Engineering College,FALSE,NA,Private Sector Training Provider,"No 2B, 28th Street, Between 71 & 72 Road, Mandalay",NA,FALSE,NA,Far East
3017982,Myanmar Metropolitan College,FALSE,NA,Higher Education Institutions,"No 232, Bargayar Street, Myay Ni Gone Sanchaung Township, Yangon",NA,FALSE,NA,Far East
3018091,AAT Business Park,FALSE,NA,Private Sector Training Provider,"AAT Business Park (3018091), No. 373, 80th Street, Between 38th & 39th Streets, Mandalay",NA,TRUE,NA,Far East
3019500,National Company for Training and Education Technologies,FALSE,NA,Private Sector Training Provider,"Saudi Arabia , Riyadh, Alquds District, Abdulrahman Alghafeqi St., Exit 11, P.O Box 272008",NA,TRUE,NA,Middle East
3020282,Britannia International Academy,FALSE,NA,Private Sector Training Provider,"64 Amangelgy street, Almaty 050000",NA,TRUE,NA,Middle East
3022935,The Way Training Institute,FALSE,NA,Private Sector Training Provider,"Asharej Area , Villa 27, Behind American School, Post Box 24240, Al Ain",NA,TRUE,NA,Middle East


In [47]:
sel_idx <- str_detect(datafiles_secondary_school_level_summary_statistics, "Contact_Details")
datafiles_secondary_school_level_summary_statistics[sel_idx]


2016 
"sta_it_402/data/demographic_data/School_Contact_Details_2018.xlsx" 
                                                               2017 
"sta_it_402/data/demographic_data/School_Contact_Details_2018.xlsx" 
                                                               2018 
"sta_it_402/data/demographic_data/School_Contact_Details_2018.xlsx"

In [48]:
summary_stats_schools <- data.frame()
summary_stats_schools


for (census_year in names(datafiles_secondary_school_level_summary_statistics[sel_idx])) {
    
    print(datafiles_secondary_school_level_summary_statistics[census_year])
    
    summary_stats_schools <- summary_stats_schools %>% 
        bind_rows(read_xlsx(datafiles_secondary_school_level_summary_statistics[census_year], 
                            sheet = datafiles_secondary_school_level_summary_statistics_sheetname[census_year], 
                            skip = 5, col_types = "text", 
                            na = c("*", "#", ".", "..", "z", "x", "N/A"), 
                            trim_ws = TRUE
                            ) %>% 
                    filter((rowSums(is.na(.)) != ncol(.)) & !is.na(as.numeric(SeedCode))) %>% 
                  
                    select(- (matches("^Primary|Secondary|Special") & #where(~ !all(is.na(as.numeric(.))))
                                    where(~ !any(str_detect(coalesce(., ""), "Primary|Secondary|Special")))
                               )) %>% 
                      
                    select(matches("authority|SeedCode|centre|name|type|address|post|year|primary|secondary|special")) %>%
                    rename_with(~ gsub("\\.+\\d+$", "", .)) %>%
                    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%
                    rename(LocalAuthority = LaName) %>%
                    mutate(across(c(Primary, Secondary, Special, matches("Address")), ~ na_if(., "0")),
                           #across(Address1, ~ gsub("(^\\$)?.+$", "\\1", .), .names = "AddressFlag"),
                           #across(Address1, ~ gsub("(^\\$)", "", .)),,
                           across(matches("Address[2-9]\\d*$"), ~ if_else((str_to_upper(.) == .), str_to_lower(.), .)),
                           across(matches("Address[2-9]\\d*$"), tools::toTitleCase),
                           
                           Year = as.integer(census_year) + 1,
                          ) %>%
                    unite("Address", matches("Address\\d"), na.rm = TRUE, sep = ",\n") %>%
                    unite("SchoolType", c(Primary, Secondary, Special), sep = ", ", na.rm = TRUE) %>%
                    filter(SchoolType != "Primary")
                 )
}

glimpse(summary_stats_schools)
unique(summary_stats_schools$SchoolType)


for (census_year in names(datafiles_secondary_school_level_summary_statistics[!sel_idx])) {
    
    print(datafiles_secondary_school_level_summary_statistics[census_year])
    
    summary_stats_schools <- summary_stats_schools %>% 
        bind_rows(read_xlsx(datafiles_secondary_school_level_summary_statistics[census_year], 
                            sheet = datafiles_secondary_school_level_summary_statistics_sheetname[census_year], 
                            skip = 1, col_types = "text", 
                            na = c("*", "#", ".", "..", "z", "x", "N/A"), 
                            trim_ws = T
                            ) %>% 
                    filter(rowSums(is.na(.)) != ncol(.)) %>% 
                    select(matches("authority|SeedCode|centre|name|type|address|post|year")) %>%
                    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%
                    mutate(Year = as.integer(census_year) + 1) %>% # records taken in Sep
                    filter(SchoolType != "Primary")
                 )
}

glimpse(summary_stats_schools)
unique(summary_stats_schools$CentreType)
unique(summary_stats_schools$SchoolType)
unique(summary_stats_schools$Year)
unique(summary_stats_schools$LocalAuthority)

<0 x 0 matrix>

                                                               2016 
"sta_it_402/data/demographic_data/School_Contact_Details_2018.xlsx" 


New names:
• `Primary` -> `Primary...11`
• `Secondary` -> `Secondary...12`
• `Special` -> `Special...13`
• `Primary` -> `Primary...17`
• `Secondary` -> `Secondary...18`
• `Special` -> `Special...19`
Warning message:
“There was 1 warning in `filter()`.
ℹ In argument: `(rowSums(is.na(.)) != ncol(.)) & !is.na(as.numeric(SeedCode))`.
Caused by warning:
! NAs introduced by coercion”


                                                               2017 
"sta_it_402/data/demographic_data/School_Contact_Details_2018.xlsx" 


New names:
• `Primary` -> `Primary...11`
• `Secondary` -> `Secondary...12`
• `Special` -> `Special...13`
• `Primary` -> `Primary...17`
• `Secondary` -> `Secondary...18`
• `Special` -> `Special...19`
Warning message:
“There was 1 warning in `filter()`.
ℹ In argument: `(rowSums(is.na(.)) != ncol(.)) & !is.na(as.numeric(SeedCode))`.
Caused by warning:
! NAs introduced by coercion”


...

                                                                        2019 
"sta_it_402/data/demographic_data/School_level_summary_statistics_2019.xlsx" 
                                                                        2020 
"sta_it_402/data/demographic_data/School_level_summary_statistics_2020.xlsx" 
                                                                        2021 
"sta_it_402/data/demographic_data/School_level_summary_statistics_2021.xlsx" 
                                                                        2022 
"sta_it_402/data/demographic_data/School_level_summary_statistics_2022.xlsx" 
Rows: 3,641
Columns: 8
$ SeedCode       <chr> "1000047", "1000144", "1000241", "1001922", "1001930", …
$ LocalAuthority <chr> "North Lanarkshire", "North Lanarkshire", "Glasgow City…
$ CentreType     <chr> "Local Authority", "Local Authority", "Local Authority"…
$ SchoolName     <chr> "Firpark Primary School", "Firpark Special School - Sec…
$ Address        <chr> "Milton St

[1] "Local Authority" "Grant-aided"     "Independent"     NA

[1] "Special"            "Secondary"          "Primary, Secondary"
[4] "Secondary, Special"

[1] 2017 2018 2019 2020 2021 2022 2023

[1] "North Lanarkshire"    "Glasgow City"         "Falkirk"             
 [4] "Edinburgh City"       "West Lothian"         "South Lanarkshire"   
 [7] "Fife"                 "Renfrewshire"         "Inverclyde"          
[10] "Highland"             "Moray"                "Aberdeenshire"       
[13] "Aberdeen City"        "Angus"                "Dundee City"         
[16] "Perth & Kinross"      "Midlothian"           "East Lothian"        
[19] "Scottish Borders"     "Clackmannanshire"     "Stirling"            
[22] "Dumfries & Galloway"  "Orkney Islands"       "Shetland Islands"    
[25] "Eilean Siar"          "Argyll & Bute"        "North Ayrshire"      
[28] "East Ayrshire"        "South Ayrshire"       "West Dunbartonshire" 
[31] "East Dunbartonshire"  "Grant-maintained"     "East Renfrewshire"   
[34] "Grant aided"          "Na h-Eileanan Siar"   "City of Edinburgh"   
[37] "Grant aided [ NOTE ]"

In [49]:
summary_stats_schools <- summary_stats_schools %>%

    rename_with(~ c("CentreStatus", "CentreType", "Postcode"), c("CentreType", "SchoolType", "PostCode")) %>%

    mutate(across(where(is.character), str_squish),
           across(where(is.character), ~ gsub("\\s*\\[?\\s?NOTE\\s?\\]?", "", ., ignore.case = TRUE, perl = TRUE)),
           across(SchoolName, ~ gsub("(^\\$)?.+$", "\\1", .), .names = "CentreClosed"),
           across(SchoolName, ~ str_replace_all(., "\\$", "")), 
           across(SchoolName, tools::toTitleCase),
           
           across(LocalAuthority, ~ if_else(str_detect(., "Edinburgh"), "City of Edinburgh", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Clackmannan"), "Clackmannanshire", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Western Isles|Eileanan|Eilean"), "Na h-Eileanan Siar", .)), 
           across(LocalAuthority, ~ gsub("\\band\\b", "\\&", .)),
           across(c(CentreStatus, LocalAuthority), ~ if_else(str_detect(., regex("^Grant(\\s|-)", ignore_case = TRUE)), 
                                            gsub("(^|[[:space:]-])([[:alpha:]])", " \\U\\2", ., perl = TRUE), .)), 
           across(c(CentreStatus, LocalAuthority), ~ str_squish(str_remove(., "Council"))),
           
           across(CentreClosed, ~ (. != "")),
           across(Year, as.ordered),
           across(where(is.character) & !LocalAuthority & !Address, as.factor),
          ) %>%
    distinct() %>%
    relocate(CentreStatus, CentreType, .after = SchoolName) %>%
    relocate(CentreClosed, .after = SchoolName) %>%
    relocate(Year, LocalAuthority) %>%

    glimpse()

unique(summary_stats_schools$LocalAuthority)
unique(summary_stats_schools$CentreStatus)

Rows: 3,641
Columns: 9
$ Year           <ord> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2…
$ LocalAuthority <chr> "North Lanarkshire", "North Lanarkshire", "Glasgow City…
$ SeedCode       <fct> 1000047, 1000144, 1000241, 1001922, 1001930, 1002732, 1…
$ SchoolName     <fct> Firpark Primary School, Firpark Special School - Second…
$ CentreClosed   <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ CentreStatus   <fct> Local Authority, Local Authority, Local Authority, Loca…
$ CentreType     <fct> "Special", "Special", "Special", "Special", "Special", …
$ Address        <chr> "Milton Street, Motherwell", "Firpark Street, Motherwel…
$ Postcode       <fct> ML1 1DH, ML1 2PR, G32 6DJ, ML5 3ET, ML5 2HT, G67 2UF, F…


[1] "North Lanarkshire"   "Glasgow City"        "Falkirk"            
 [4] "City of Edinburgh"   "West Lothian"        "South Lanarkshire"  
 [7] "Fife"                "Renfrewshire"        "Inverclyde"         
[10] "Highland"            "Moray"               "Aberdeenshire"      
[13] "Aberdeen City"       "Angus"               "Dundee City"        
[16] "Perth & Kinross"     "Midlothian"          "East Lothian"       
[19] "Scottish Borders"    "Clackmannanshire"    "Stirling"           
[22] "Dumfries & Galloway" "Orkney Islands"      "Shetland Islands"   
[25] "Na h-Eileanan Siar"  "Argyll & Bute"       "North Ayrshire"     
[28] "East Ayrshire"       "South Ayrshire"      "West Dunbartonshire"
[31] "East Dunbartonshire" "Grant Maintained"    "East Renfrewshire"  
[34] "Grant Aided"

[1] Local Authority Grant Aided     Independent     <NA>           
Levels: Grant Aided Independent Local Authority

In [50]:
summary_stats_schools %>%
    count(CentreStatus, CentreType)

CentreStatus,CentreType,n
<fct>,<fct>,<int>
Grant Aided,"Primary, Secondary",3
Grant Aided,Special,21
Independent,"Primary, Secondary",141
Independent,Secondary,21
Independent,"Secondary, Special",1
Independent,Special,115
Local Authority,"Primary, Secondary",73
Local Authority,Secondary,1001
Local Authority,Special,387


In [51]:
summary_stats_schools <- summary_stats_schools %>% 
    mutate(#across(LocalAuthority, ~ fct_relevel(., !!!(summary_stats_schools %>%
           #                                             filter(str_detect(LocalAuthority, regex("Grant", ignore_case = TRUE))) %>%
           #                                             distinct(LocalAuthority) %>%
           #                                             arrange(LocalAuthority) %>%
           #                                             deframe()), after = Inf)), 
           across(LocalAuthority, ~ fct_collapse(., `Grant Aided` = (summary_stats_schools %>%
                                                                            filter(str_detect(LocalAuthority, regex("^Grant", ignore_case = TRUE))) %>%
                                                                            distinct(LocalAuthority) %>%
                                                                            arrange(LocalAuthority) %>%
                                                                            deframe() %>%
                                                                            as.character()))),
           across(LocalAuthority, ~ fct_relevel(., "Grant Aided", after = Inf)), 
          )

levels(summary_stats_schools$LocalAuthority)

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "Falkirk"             "Fife"                "Glasgow City"       
[16] "Highland"            "Inverclyde"          "Midlothian"         
[19] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[22] "North Lanarkshire"   "Orkney Islands"      "Perth & Kinross"    
[25] "Renfrewshire"        "Scottish Borders"    "Shetland Islands"   
[28] "South Ayrshire"      "South Lanarkshire"   "Stirling"           
[31] "West Dunbartonshire" "West Lothian"        "Grant Aided"

In [52]:
school_contact_list_2023 <- read_xlsx(paste(path_to_file_store, "demographic_data", "school_contact_list_30_April_2023.xlsx", sep = "/"),
          trim_ws = TRUE, 
          sheet = "Open Schools", col_types = "text",
          skip = 5, 
          na = c("*", "#", ".", "..", "z", "x", "N/A"), 
         ) %>% 
    filter(rowSums(is.na(.)) != ncol(.)) %>% 
    select(- matches("UPRN|Email|Phone|Website")) %>% 
 
    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%
    rename_with(~ c("LocalAuthority", "CentreStatus", "Postcode"), c("LaName", "CentreType", "PostCode")) %>%


    mutate(across(where(is.character), str_squish),
           
           across(matches("Address"), ~ na_if(., "0")),
           across(matches("Address\\w*[2-9]\\d*$"), ~ if_else((str_to_upper(.) == .), str_to_lower(.), .)),
           across(matches("Address\\w*[2-9]\\d*$"), tools::toTitleCase),
                           
           across(matches("PreSchool|Primary|Secondary|Special|Gaelic"), ~ if_else((. == "Yes"), cur_column(), NA_character_)),
           across(matches("PreSchool|Primary|Secondary|Special"), ~ str_remove(., "Department")),
           across(matches("Gaelic|.+Special"), ~ !is.na(.)),
           
           across(SchoolName, tools::toTitleCase),
           across(LocalAuthority, ~ if_else(str_detect(., "Edinburgh"), "City of Edinburgh", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Clackmannan"), "Clackmannanshire", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Western Isles|Eileanan|Eilean"), "Na h-Eileanan Siar", .)), 
           across(LocalAuthority, ~ gsub("\\band\\b", "\\&", .)),
           across(c(CentreStatus, LocalAuthority), ~ if_else(str_detect(., regex("^Grant(\\s|-)", ignore_case = TRUE)), 
                                            gsub("(^|[[:space:]-])([[:alpha:]])", " \\U\\2", ., perl = TRUE), .)), 
           across(c(CentreStatus, LocalAuthority), ~ str_squish(str_remove(., "Council"))),

           Year = as.integer(2023), # ™†odo - chnge to pattern
          ) %>%

    unite("Address", matches("Address\\w*\\d"), na.rm = TRUE, sep = ",\n") %>%
    unite("CentreType", matches("^(PreSchool|Primary|Secondary|Special)"), sep = ", ", na.rm = TRUE) %>%
    filter(str_detect(CentreType, "Secondary|Special")) %>%
    mutate(across(Address, str_squish),
           across(where(is.character) & !Address, as.factor)) %>%

    relocate(CentreStatus, CentreType, .after = SchoolName) %>%
    relocate(Address, Postcode, .after = last_col()) %>%
    relocate(Year) %>%

    distinct() %>%
    glimpse()

Rows: 486
Columns: 11
$ Year                  <int> 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, …
$ SeedCode              <fct> 1004140, 1004166, 5234034, 5235634, 5235839, 524…
$ LocalAuthority        <fct> Aberdeen City, Aberdeen City, Aberdeen City, Abe…
$ SchoolName            <fct> Orchard Brae School, Lochside Academy, Bucksburn…
$ CentreStatus          <fct> Local Authority, Local Authority, Local Authorit…
$ CentreType            <fct> "Special", "Secondary", "Secondary", "Secondary"…
$ GaelicUnit            <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ IntegratedSpecialUnit <lgl> FALSE, FALSE, TRUE, TRUE, TRUE, FALSE, TRUE, TRU…
$ Denomination          <fct> Non-denominational, Non-denominational, Non-deno…
$ Address               <chr> "Howes Road, Aberdeen", "Wellington Circle, Alte…
$ Postcode              <fct> AB16 7RH, AB12 3JG, AB21 9DG, AB22 8RR, AB15 9TP…


In [53]:
school_contact_list_2023 <- school_contact_list_2023 %>% 
    mutate(across(LocalAuthority, ~ fct_collapse(., `Grant Aided` = (school_contact_list_2023 %>%
                                                                            filter(str_detect(LocalAuthority, regex("^Grant", ignore_case = TRUE))) %>%
                                                                            distinct(LocalAuthority) %>%
                                                                            arrange(LocalAuthority) %>%
                                                                            deframe() %>%
                                                                            as.character()))),
           across(LocalAuthority, ~ fct_relevel(., "Grant Aided", after = Inf)), 
          )

levels(school_contact_list_2023$LocalAuthority)

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "Falkirk"             "Fife"                "Glasgow City"       
[16] "Highland"            "Inverclyde"          "Midlothian"         
[19] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[22] "North Lanarkshire"   "Orkney Islands"      "Perth & Kinross"    
[25] "Renfrewshire"        "Scottish Borders"    "Shetland Islands"   
[28] "South Ayrshire"      "South Lanarkshire"   "Stirling"           
[31] "West Dunbartonshire" "West Lothian"        "Grant Aided"

In [54]:
n_distinct(school_contact_list_2023$SeedCode)

school_contact_list_2023 %>%
    group_by(SeedCode) %>%
    add_tally() %>%
    filter(n > 1)

[1] 486

Year,SeedCode,LocalAuthority,SchoolName,CentreStatus,CentreType,GaelicUnit,IntegratedSpecialUnit,Denomination,Address,Postcode,n
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<lgl>,<lgl>,<fct>,<chr>,<fct>,<int>


In [55]:
school_rolls_1996_2018 <- read_xlsx(paste(path_to_file_store, "demographic_data", "school-rolls-1996-2018.xlsx", sep = "/"),
                                      trim_ws = TRUE, 
                                      sheet = "Data", col_types = "text"
                                     ) %>% 
    filter(rowSums(is.na(.)) != ncol(.)) %>% 
 
    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%
    rename_with(~ c("SeedCode", "LocalAuthority", "SchoolName", "CentreStatus", "CentreType"), 
                c("Code", "LaName", "Name", "SchoolType", "Sector")) %>%

    select(- matches("^\\d{4}$|LaCode")) %>% 
    filter(CentreType != "Primary") %>%

    mutate(across(where(is.character), str_squish),
           across(SchoolName, tools::toTitleCase),
           
           across(LocalAuthority, ~ if_else(str_detect(., "Edinburgh"), "City of Edinburgh", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Clackmannan"), "Clackmannanshire", .)), 
           across(LocalAuthority, ~ if_else(str_detect(., "Western Isles|Eileanan|Eilean"), "Na h-Eileanan Siar", .)), 
           across(LocalAuthority, ~ gsub("\\band\\b", "\\&", .)),
           across(c(CentreStatus, LocalAuthority), ~ if_else(str_detect(., regex("^Grant(\\s|-)", ignore_case = TRUE)), 
                                            gsub("(^|[[:space:]-])([[:alpha:]])", " \\U\\2", ., perl = TRUE), .)), 
           across(c(CentreStatus, LocalAuthority), ~ str_squish(str_remove(., "Council"))),
           
           across(where(is.character), as.factor),
          ) %>%
    distinct() %>%

    glimpse()

Rows: 819
Columns: 5
$ LocalAuthority <fct> Aberdeen City, Aberdeen City, Aberdeen City, Aberdeen C…
$ SeedCode       <fct> 5244439, 5235731, 5235634, 5234034, 5235839, 5243335, 5…
$ SchoolName     <fct> "Aberdeen Grammar", "Bankhead Academy", "Bridge of Don …
$ CentreType     <fct> Secondary, Secondary, Secondary, Secondary, Secondary, …
$ CentreStatus   <fct> Local Authority, Local Authority, Local Authority, Loca…


In [56]:
school_rolls_1996_2018 <- school_rolls_1996_2018 %>% 
    mutate(across(LocalAuthority, ~ fct_collapse(., `Grant Aided` = (school_rolls_1996_2018 %>%
                                                                            filter(str_detect(LocalAuthority, regex("^Grant", ignore_case = TRUE))) %>%
                                                                            distinct(LocalAuthority) %>%
                                                                            arrange(LocalAuthority) %>%
                                                                            deframe() %>%
                                                                            as.character()))),
           across(LocalAuthority, ~ fct_relevel(., "Grant Aided", after = Inf)), 
          )

levels(school_rolls_1996_2018$LocalAuthority)

[1] "Aberdeen City"       "Aberdeenshire"       "Angus"              
 [4] "Argyll & Bute"       "City of Edinburgh"   "Clackmannanshire"   
 [7] "Dumfries & Galloway" "Dundee City"         "East Ayrshire"      
[10] "East Dunbartonshire" "East Lothian"        "East Renfrewshire"  
[13] "Falkirk"             "Fife"                "Glasgow City"       
[16] "Highland"            "Inverclyde"          "Midlothian"         
[19] "Moray"               "Na h-Eileanan Siar"  "North Ayrshire"     
[22] "North Lanarkshire"   "Orkney Islands"      "Perth & Kinross"    
[25] "Renfrewshire"        "Scottish Borders"    "Shetland Islands"   
[28] "South Ayrshire"      "South Lanarkshire"   "Stirling"           
[31] "West Dunbartonshire" "West Lothian"        "Grant Aided"

In [57]:
n_distinct(school_rolls_1996_2018$SeedCode)

school_rolls_1996_2018 %>%
    group_by(SeedCode) %>%
    add_tally() %>%
    filter(n > 1)

[1] 818

LocalAuthority,SeedCode,SchoolName,CentreType,CentreStatus,n
<fct>,<fct>,<fct>,<fct>,<fct>,<int>
Dundee City,5320038,Off-Site Education Service,Secondary,Local Authority,2
Dundee City,5320038,Off-Site Education,Special,Local Authority,2


In [58]:
glimpse(school_contact_list_2023)

Rows: 486
Columns: 11
$ Year                  <int> 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, …
$ SeedCode              <fct> 1004140, 1004166, 5234034, 5235634, 5235839, 524…
$ LocalAuthority        <fct> Aberdeen City, Aberdeen City, Aberdeen City, Abe…
$ SchoolName            <fct> Orchard Brae School, Lochside Academy, Bucksburn…
$ CentreStatus          <fct> Local Authority, Local Authority, Local Authorit…
$ CentreType            <fct> "Special", "Secondary", "Secondary", "Secondary"…
$ GaelicUnit            <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ IntegratedSpecialUnit <lgl> FALSE, FALSE, TRUE, TRUE, TRUE, FALSE, TRUE, TRU…
$ Denomination          <fct> Non-denominational, Non-denominational, Non-deno…
$ Address               <chr> "Howes Road, Aberdeen", "Wellington Circle, Alte…
$ Postcode              <fct> AB16 7RH, AB12 3JG, AB21 9DG, AB22 8RR, AB15 9TP…


In [59]:
summary_stats_schools %>%
    filter(SeedCode %in% c("5780039", "5280036")) %>%
    distinct(Year, SeedCode, SchoolName, Address, Postcode, CentreType, LocalAuthority) %>%
    arrange(desc(Year))

summary_stats_schools %>%
    filter(str_detect(SchoolName, "Focus|Springvale|Oakwood|Oneschool|Global|Ochil") | str_detect(Address, "Focus|Springvale"))

centre_postcodes %>%
    filter(Centre %in% c("5780039", "5280036")) %>%
    distinct(Centre, CentreName, CentreType, Address, Postcode)

centre_postcodes %>%
    filter(str_detect(CentreName, "Focus|Springvale|Oakwood|Oneschool|Global|Ochil|Caledonia") | 
           str_detect(Address, "Focus|Springvale|Millden"))

Year,SeedCode,SchoolName,Address,Postcode,CentreType,LocalAuthority
<ord>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>
2019,5780039,Focus School Caledonia Campus,"The Pavilions, Alloa West Business Park, Alloa",FK10 1TA,"Primary, Secondary",Falkirk
2018,5780039,Focus School Caledonia Campus,"The Pavilions, Alloa West Business Park, Alloa",FK10 1TA,"Primary, Secondary",Falkirk
2017,5280036,Focus School - Millden Campus,"Springvale Education Trust, Castle Way, Ellon",AB41 9RG,"Primary, Secondary",Aberdeenshire
2017,5780039,Focus School - Ochil Campus,"The Pavilions, Alloa West Business Park, Alloa",FK10 1TA,"Primary, Secondary",Falkirk


Year,LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,Address,Postcode
<ord>,<fct>,<fct>,<fct>,<lgl>,<fct>,<fct>,<chr>,<fct>
2017,Aberdeenshire,5280036,Focus School - Millden Campus,FALSE,Independent,"Primary, Secondary","Springvale Education Trust, Castle Way, Ellon",AB41 9RG
2017,Perth & Kinross,5381142,Ochil Tower School,FALSE,Independent,Special,"140 High Street, Auchterarder, Perthshire",PH3 1AD
2017,Falkirk,5780039,Focus School - Ochil Campus,FALSE,Independent,"Primary, Secondary","The Pavilions, Alloa West Business Park, Alloa",FK10 1TA
2018,Perth & Kinross,5381142,Ochil Tower School,FALSE,Independent,Special,"140 High Street, Auchterarder, Perthshire",PH3 1AD
2018,Falkirk,5780039,Focus School Caledonia Campus,FALSE,Independent,"Primary, Secondary","The Pavilions, Alloa West Business Park, Alloa",FK10 1TA
2019,Falkirk,5780039,Focus School Caledonia Campus,FALSE,Independent,"Primary, Secondary","The Pavilions, Alloa West Business Park, Alloa",FK10 1TA
2019,Perth & Kinross,5381142,Ochil Tower School,FALSE,Independent,Special,"140 High Street, Auchterarder, Perthshire",PH3 1AD


Centre,CentreName,CentreType,Address,Postcode
<fct>,<chr>,<fct>,<chr>,<fct>


Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>
3021912,Focus College,FALSE,NA,Private Sector Training Provider,"10 - 1753 Dolphin Avenue, Kelowna, British Columbia",V1Y 8A6,TRUE,NA,Other
3025039,One Global Education FZ-LLC,FALSE,NA,Private Sector Training Provider,"Behind RAK Police Headquarters, RAKEZ Academic Zone, Al Dhait South, Ras Al Khaimah",41222,FALSE,NA,Middle East
5381142,Ochil Tower School,FALSE,Independent,Special School,"140 High Street, Auchterarder",PH3 1AD,FALSE,NA,Perth & Kinross


In [60]:
summary_stats_schools %>%
    filter(SeedCode %in% c("5632838")) %>%
    distinct(Year, SeedCode, SchoolName, Address, Postcode, CentreType, LocalAuthority) %>%
    arrange(desc(Year))

Year,SeedCode,SchoolName,Address,Postcode,CentreType,LocalAuthority
<ord>,<fct>,<fct>,<chr>,<fct>,<fct>,<fct>
2023,5632838,Kelso High School,NA,NA,Secondary,Scottish Borders
2022,5632838,Kelso High School,NA,NA,Secondary,Scottish Borders
2021,5632838,Kelso High School,NA,NA,Secondary,Scottish Borders
2020,5632838,Kelso High School,NA,NA,Secondary,Scottish Borders
2019,5632838,Kelso High School,"Angraflat Road, Kelso, Roxburghshire",TD5 7NL,Secondary,Scottish Borders
2018,5632838,Kelso High School,"Bowmont Street, Kelso, Roxburghshire",TD5 7EG,Secondary,Scottish Borders
2017,5632838,Kelso High School,"Bowmont Street, Kelso, Roxburghshire",TD5 7EG,Secondary,Scottish Borders


Centre,CentreName,CentreType,Address,Postcode
<fct>,<chr>,<fct>,<chr>,<fct>
5632838,Kelso High School,Secondary School,"Angraflat Road, Kelso",TD5 7NL


In [62]:
summary_stats_schools %>%
    filter(SeedCode == "5235634") %>%
    select(- Year) %>%
    group_by(Address, Postcode) %>%
    add_tally() 

LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,Address,Postcode,n
<fct>,<fct>,<fct>,<lgl>,<fct>,<fct>,<chr>,<fct>,<int>
Aberdeen City,5235634,Bridge of Don Academy,FALSE,Local Authority,Secondary,"Braehead Way, Bridge of Don, Aberdeen",AB22 8RR,3
Aberdeen City,5235634,Bridge of Don Academy,FALSE,Local Authority,Secondary,"Braehead Way, Bridge of Don, Aberdeen",AB22 8RR,3
Aberdeen City,5235634,Bridge of Don Academy,FALSE,Local Authority,Secondary,"Braehead Way, Bridge of Don, Aberdeen",AB22 8RR,3
Aberdeen City,5235634,Bridge of Don Academy,FALSE,NA,Secondary,NA,NA,4
Aberdeen City,5235634,Bridge of Don Academy,FALSE,NA,Secondary,NA,NA,4
Aberdeen City,5235634,Bridge of Don Academy,FALSE,NA,Secondary,NA,NA,4
Aberdeen City,5235634,Bridge of Don Academy,FALSE,NA,Secondary,NA,NA,4


In [63]:
summary_stats_schools %>%
    filter(SeedCode == "8442746")

Year,LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,Address,Postcode
<ord>,<fct>,<fct>,<fct>,<lgl>,<fct>,<fct>,<chr>,<fct>
2017,Glasgow City,8442746,St Kevin's Primary School,FALSE,Local Authority,Special,"25 Fountainwell Road, Glasgow",G21 1TN
2018,Glasgow City,8442746,St Kevin's Primary School,FALSE,Local Authority,Special,"25 Fountainwell Road, Glasgow",G21 1TN
2019,Glasgow City,8442746,St Kevin's Primary School,FALSE,Local Authority,Special,"22 Pinkston Drive, Glasgow",G21 1NL
2020,Glasgow City,8442746,St Kevin's Primary School,FALSE,NA,Special,NA,NA
2021,Glasgow City,8442746,St Kevin's Primary School,FALSE,NA,Special,NA,NA
2022,Glasgow City,8442746,St Kevin's Primary School,FALSE,NA,Special,NA,NA
2023,Glasgow City,8442746,St Kevin's Primary School,FALSE,NA,Special,NA,NA


In [64]:
dim(summary_stats_schools)

summary_stats_schools %>%
    filter(!if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%

    bind_rows(summary_stats_schools %>%
                filter(!if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%
                select(- Year) %>%

                right_join(summary_stats_schools %>%
                               filter(if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%
                               select(- c(CentreStatus, Address, Postcode)), 
                           by = c("LocalAuthority", "SeedCode", "SchoolName", "CentreClosed", "CentreType")
                )
             ) %>%

    distinct(across(!Year)) %>%
    group_by(SeedCode) %>%
    add_tally() %>%
    filter(n > 1) %>%
    arrange(SeedCode, desc(n)) 

[1] 3641    9

Warning message in right_join(., summary_stats_schools %>% filter(if_all(c(CentreStatus, :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 345 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"`
  to silence this warning.”


LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,Address,Postcode,n
<fct>,<fct>,<fct>,<lgl>,<fct>,<fct>,<chr>,<fct>,<int>
North Lanarkshire,1000144,Firpark Special School - Secondary,FALSE,Local Authority,Special,"Firpark Street, Motherwell",ML1 2PR,2
North Lanarkshire,1000144,Firpark Secondary School,FALSE,NA,Special,NA,NA,2
Falkirk,1002848,Mariner Support Service,FALSE,Local Authority,Special,"Bog Road Industrial Estate, Bog Road, Falkirk",FK2 9PB,3
Falkirk,1002848,Mariner Support Services,FALSE,NA,Special,NA,NA,3
Falkirk,1002848,Inclusion & Wellbeing Service,FALSE,NA,Special,NA,NA,3
West Lothian,1003038,Connolly Campus School,FALSE,Local Authority,Special,"Hopefield Road, Blackburn",EH47 7HZ,2
West Lothian,1003038,Connolly Campus School,FALSE,Local Authority,Special,"Hopefield Road, Blackburn, West Lothian",EH47 7HZ,2
Renfrewshire,1003658,Riverbrae School,FALSE,Local Authority,Special,"Brediland Road, Linwood",PA3 3RX,2
Renfrewshire,1003658,Riverbrae School,FALSE,Local Authority,Special,"2 Middleton Road, Linwood",PA3 3DP,2


In [65]:
summary_stats_schools <- summary_stats_schools %>%
    mutate(GrantAided = (CentreStatus == "Grant Aided"),
           across(CentreStatus, ~ fct_recode(., `Education Authority` = "Local Authority", NULL = "Grant Aided")),
          ) %>%
    relocate(GrantAided, .after = CentreType) %>%
    glimpse()

summary_stats_schools %>%
    count(GrantAided, CentreStatus)



Rows: 3,641
Columns: 10
$ Year           <ord> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2…
$ LocalAuthority <fct> North Lanarkshire, North Lanarkshire, Glasgow City, Nor…
$ SeedCode       <fct> 1000047, 1000144, 1000241, 1001922, 1001930, 1002732, 1…
$ SchoolName     <fct> Firpark Primary School, Firpark Special School - Second…
$ CentreClosed   <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ CentreStatus   <fct> Education Authority, Education Authority, Education Aut…
$ CentreType     <fct> "Special", "Special", "Special", "Special", "Special", …
$ GrantAided     <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ Address        <chr> "Milton Street, Motherwell", "Firpark Street, Motherwel…
$ Postcode       <fct> ML1 1DH, ML1 2PR, G32 6DJ, ML5 3ET, ML5 2HT, G67 2UF, F…


GrantAided,CentreStatus,n
<lgl>,<fct>,<int>
FALSE,Independent,278
FALSE,Education Authority,1461
TRUE,NA,24
NA,NA,1878


In [66]:
dim(summary_stats_schools)

summary_stats_schools <- summary_stats_schools %>%
    filter(!if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%

    bind_rows(summary_stats_schools %>%
                filter(!if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%
                select(- Year) %>%

                right_join(summary_stats_schools %>%
                               filter(if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%
                               select(- c(CentreStatus, Address, Postcode)), 
                           by = c("LocalAuthority", "SeedCode", "SchoolName", "CentreClosed", "CentreType", "GrantAided")
                )
             ) %>%

    distinct() %>%
    glimpse()

[1] 3641   10

Rows: 3,641
Columns: 10
$ Year           <ord> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2…
$ LocalAuthority <fct> North Lanarkshire, North Lanarkshire, Glasgow City, Nor…
$ SeedCode       <fct> 1000047, 1000144, 1000241, 1001922, 1001930, 1002732, 1…
$ SchoolName     <fct> Firpark Primary School, Firpark Special School - Second…
$ CentreClosed   <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ CentreStatus   <fct> Education Authority, Education Authority, Education Aut…
$ CentreType     <fct> "Special", "Special", "Special", "Special", "Special", …
$ GrantAided     <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ Address        <chr> "Milton Street, Motherwell", "Firpark Street, Motherwel…
$ Postcode       <fct> ML1 1DH, ML1 2PR, G32 6DJ, ML5 3ET, ML5 2HT, G67 2UF, F…


In [67]:

setdiff(summary_stats_schools$LocalAuthority, centre_postcodes$GeoArea)
setdiff(centre_postcodes$GeoArea, summary_stats_schools$LocalAuthority)


setdiff(summary_stats_schools$SeedCode, centre_postcodes$Centre)
setdiff(centre_postcodes$Centre, summary_stats_schools$SeedCode)

[1] "Grant Aided"

[1] "Middle East" "Far East"    "Wales"       "England"     "Other"      
[6] "South Asia"

[1] "1000047" "1000241" "1001922" "1002848" "1002945" "1003038" "5119332"
  [8] "5120748" "5130239" "5135834" "5142431" "5142539" "5240042" "5245044"
 [15] "5245443" "5246040" "5408946" "5427649" "5500346" "5500443" "5500648"
 [22] "5502640" "5536049" "5536243" "5700248" "5702941" "5720540" "5720648"
 [29] "5726840" "5740045" "5747244" "5940842" "5948630" "5948738" "6003036"
 [36] "6003230" "6003834" "6104134" "8109931" "8111030" "8111448" "8223939"
 [43] "8224242" "8240140" "8242143" "8315248" "8338744" "8350647" "8414041"
 [50] "8421544" "8430640" "8440042" "8440344" "8440441" "8440549" "8441049"
 [57] "8441146" "8441448" "8442746" "8443440" "8447241" "8459045" "8459142"
 [64] "8506736" "8507848" "8517231" "8518246" "8518343" "8534144" "8534543"
 [71] "8550743" "8550840" "1003364" "1070037" "1070045" "1070134" "1070169"
 [78] "1070231" "1070436" "5280036" "5280044" "5280141" "5280338" "5280737"
 [85] "5280834" "5380235" "5380537" "5381037" "5381649" "5381940" "5480043"
 [92] "5480434" "5480647" "5571448" "5580048" "5580439" "5580730" "5581931"
 [99] "5582237" "5582334" "5582636" "5583330" "5584221" "5584647" "5680131"
[106] "5780039" "5780942" "5781043" "5980240" "5980445" "8280240" "8370044"
[113] "8400245" "8480044" "8482330" "8580545" "8680639" "5245842" "5546346"
[120] "5600146" "8110948" "8400946" "5121442" "1003062" "1004409" "8400440"
[127] "1004590" "1005979"

[1] "1002759" "1002856" "1002953" "1003054" "1003151" "1003259" "1003356"
  [8] "1003453" "1003550" "1004891" "2094029" "2094274" "2094363" "2294176"
 [15] "2494086" "2494108" "3000338" "3000389" "3000435" "3000451" "3000532"
 [22] "3000753" "3001008" "3001024" "3001105" "3001121" "3001229" "3001342"
 [29] "3001377" "3001415" "3002071" "3002128" "3002225" "3002233" "3002292"
 [36] "3002322" "3002349" "3002438" "3002535" "3002616" "3002624" "3002837"
 [43] "3003043" "3003051" "3003132" "3003191" "3003299" "3003302" "3003590"
 [50] "3003604" "3003647" "3003817" "3004279" "3004325" "3004341" "3004457"
 [57] "3004597" "3004740" "3004767" "3004856" "3004880" "3005038" "3005046"
 [64] "3005151" "3005178" "3005194" "3005267" "3005348" "3005356" "3005372"
 [71] "3005380" "3005399" "3005402" "3005437" "3005445" "3005453" "3005593"
 [78] "3005860" "3005917" "3006026" "3006050" "3006166" "3006557" "3006654"
 [85] "3006670" "3006735" "3006840" "3006891" "3006980" "3006999" "3007103"
 [92] "3007111" "3007243" "3007391" "3007421" "3007499" "3007510" "3007553"
 [99] "3007596" "3007685" "3007715" "3007790" "3007847" "3008037" "3008479"
[106] "3008495" "3008576" "3008703" "3008754" "3008789" "3009084" "3009130"
[113] "3009157" "3009181" "3009416" "3009467" "3009505" "3009548" "3009564"
[120] "3010112" "3010120" "3010139" "3010163" "3010279" "3010384" "3010392"
[127] "3010422" "3010589" "3010864" "3010910" "3010961" "3011046" "3011089"
[134] "3011135" "3011143" "3011151" "3011178" "3011216" "3011267" "3011275"
[141] "3011348" "3011402" "3011437" "3011534" "3011623" "3011666" "3011690"
[148] "3011739" "3011917" "3012034" "3012174" "3012212" "3012263" "3012271"
[155] "3012328" "3012336" "3012360" "3012441" "3012476" "3012557" "3012603"
[162] "3012654" "3012700" "3012727" "3012743" "3012808" "3012883" "3013014"
[169] "3013103" "3013197" "3013219" "3013278" "3013308" "3013332" "3013456"
[176] "3013588" "3013642" "3013707" "3013839" "3013987" "3014010" "3014088"
[183] "3014207" "3014339" "3014371" "3015998" "3016013" "3016072" "3016455"
[190] "3016463" "3016684" "3016722" "3016757" "3016897" "3017060" "3017079"
[197] "3017109" "3017435" "3017451" "3017524" "3017591" "3017648" "3017729"
[204] "3017737" "3017850" "3017915" "3017923" "3017982" "3018083" "3018091"
[211] "3018210" "3018237" "3018288" "3018369" "3018466" "3018873" "3018970"
[218] "3019047" "3019063" "3019136" "3019217" "3019292" "3019446" "3019500"
[225] "3019551" "3019616" "3019640" "3019799" "3019888" "3019950" "3019993"
[232] "3020002" "3020142" "3020177" "3020215" "3020282" "3020312" "3020428"
[239] "3020452" "3020487" "3020509" "3020541" "3020576" "3020584" "3020614"
[246] "3020622" "3020665" "3020681" "3020703" "3020746" "3020797" "3020843"
[253] "3020851" "3020908" "3020959" "3021033" "3021076" "3021130" "3021211"
[260] "3021289" "3021386" "3021424" "3021491" "3021572" "3021599" "3021629"
[267] "3021645" "3021653" "3021661" "3021696" "3021742" "3021807" "3021882"
[274] "3021912" "3022005" "3022021" "3022056" "3022064" "3022102" "3022218"
[281] "3022226" "3022242" "3022323" "3022439" "3022749" "3022773" "3022803"
[288] "3022811" "3022838" "3022870" "3022935" "3023087" "3023141" "3023184"
[295] "3023230" "3023281" "3023400" "3023427" "3023486" "3023494" "3023540"
[302] "3023621" "3023664" "3023680" "3023737" "3023877" "3023982" "3024016"
[309] "3024075" "3024105" "3024113" "3024172" "3024180" "3024334" "3024385"
[316] "3024423" "3024547" "3024644" "3024709" "3024806" "3024946" "3025039"
[323] "3025063" "3025098" "3025187" "3025209" "3025284" "3025330" "3025608"
[330] "3025632" "3025802" "3025810" "3025853" "3026035" "3026140" "3026191"
[337] "3026272" "3026302" "3026388" "3026469" "3026485" "3026647" "3026752"
[344] "3026930" "4194039" "4194446" "4194616" "4194640" "4194691" "4194721"
[351] "4194780" "4195191" "4195213" "4195426" "4195612" "4195671" "4195841"
[358] "4196074" "4196104" "4196147" "4196325" "4196368" "4196694" "4196775"
[365] "4196945" "4196996" "4197186" "4197267" "419

In [68]:
summary_stats_schools <- summary_stats_schools %>%
    filter(!if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%

    bind_rows(summary_stats_schools %>%
                filter(if_all(c(CentreStatus, Address, Postcode), ~ is.na(.) | (. == ""))) %>%
                select(- c(CentreStatus, Address, Postcode)) %>%

                left_join(centre_postcodes %>%
                              distinct(Centre, CentreStatus, Address, Postcode), 
                          by = c("SeedCode" = "Centre")
                         ) 
              ) %>%

    distinct() %>%
    glimpse()

summary_stats_schools %>%
    filter(if_any(c(CentreStatus, Postcode, Address), is.na))

Rows: 3,641
Columns: 10
$ Year           <ord> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2…
$ LocalAuthority <fct> North Lanarkshire, North Lanarkshire, Glasgow City, Nor…
$ SeedCode       <fct> 1000047, 1000144, 1000241, 1001922, 1001930, 1002732, 1…
$ SchoolName     <fct> Firpark Primary School, Firpark Special School - Second…
$ CentreClosed   <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ CentreStatus   <fct> Education Authority, Education Authority, Education Aut…
$ CentreType     <fct> "Special", "Special", "Special", "Special", "Special", …
$ GrantAided     <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE,…
$ Address        <chr> "Milton Street, Motherwell", "Firpark Street, Motherwel…
$ Postcode       <fct> ML1 1DH, ML1 2PR, G32 6DJ, ML5 3ET, ML5 2HT, G67 2UF, F…


Year,LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,GrantAided,Address,Postcode
<ord>,<fct>,<fct>,<fct>,<lgl>,<fct>,<fct>,<lgl>,<chr>,<fct>
2017,Grant Aided,8470332,Jordanhill School,FALSE,NA,"Primary, Secondary",TRUE,"45 Chamberlain Road, Jordanhill, Glasgow",G13 1SP
2017,Grant Aided,5570948,Donaldson's School,FALSE,NA,Special,TRUE,"Preston Road, Linlithgow, West Lothian",EH49 6HZ
2017,Grant Aided,5571146,Royal Blind School,FALSE,NA,Special,TRUE,"2b Craigmillar Park, Edinburgh",EH16 5NA
2017,Grant Aided,5571448,Harmeny Education Trust,FALSE,NA,Special,TRUE,"45 Mansfield Road, Balerno, Midlothian",EH14 7JY
2017,Grant Aided,8370044,Craighalbert-Scottish Centre for Children with Motor Impairments,FALSE,NA,Special,TRUE,"Craighalbert Centre, 1 Craighalbert Way, Cumbernauld",G68 0LS
2017,Grant Aided,8471045,East Park School,FALSE,NA,Special,TRUE,"1092 Maryhill Road, Glasgow",G20 9TD
2017,Grant Aided,8570140,Stanmore House School,FALSE,NA,Special,TRUE,Lanark,ML11 7RR
2017,Grant Aided,8670242,Corseford Residential School,FALSE,NA,Special,TRUE,"Milliken Park - Howwood Road, Kilbarchan, Renfrewshire",PA10 2NT
2018,Grant Aided,5570948,Donaldson's School,FALSE,NA,Special,TRUE,"Preston Road, Linlithgow, West Lothian",EH49 6HZ


In [69]:
nrow(summary_stats_schools)

summary_stats_schools %>%
    distinct(SeedCode) %>%
    nrow()

summary_stats_schools %>%
    filter(is.na(Postcode)) %>%
    distinct(SeedCode, SchoolName) %>%
    nrow()

summary_stats_schools %>%
    filter(!is.na(Postcode)) %>%
    distinct(SeedCode, SchoolName, Postcode) %>%
    nrow()

[1] 3641

[1] 605

[1] 60

[1] 714

#### END &ndash; manually fill in remaining blanks and correct errors in postcodes and addresses

### Merge all demographics 

* correct address data
* fill in and normalise geo metadata
* flag differences in names

In [70]:
centre_postcodes %>%
    glimpse()

summary_stats_schools %>%
    select(- c(Year, GrantAided)) %>%
    glimpse()

school_contact_list_2023 %>%
    select(- c(Year, GaelicUnit, IntegratedSpecialUnit, Denomination)) %>%
    glimpse()

school_rolls_1996_2018 %>%
    glimpse()

Rows: 987
Columns: 10
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <fct> Education Authority, Education Authority, Education…
$ CentreType         <fct> Special School, Special School, Secondary School, F…
$ Address            <chr> "Firpark Street,\nMotherwell", "67 Townhead Road,\n…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ GeoArea            <fct> North Lanarkshire, North Lanarkshire, North Lanarks…
Rows: 3,641
Columns: 8
$ LocalAuthority <fct> North Lanarkshire, North Lanarkshire, Glasgow City, Nor…
$ SeedCode       <fct> 1000047, 1000144, 1000241, 1001922, 1001930, 1002732

In [681]:
tmp_df <- summary_stats_schools %>%
    select(- c(Year, GrantAided)) %>%

    full_join(school_contact_list_2023 %>%
                  select(- any_of(c("Year", "GaelicUnit", "IntegratedSpecialUnit", "Denomination"))) %>% 
                  mutate(across(CentreStatus, ~ fct_recode(., "Education Authority" = "Local Authority"))) %>%
                  rename_with(~ paste0(., "_fill"), 
                              !any_of(c("LocalAuthority", "SeedCode", "SchoolName", "CentreStatus", "Address", "Postcode")))
             ) %>%

    rowwise() %>%
    mutate(across(matches("CentreType"), ~ str_split(., pattern = "\\s*,\\s*")),
           CentreType = unique(c_across(matches("CentreType"))) %>% 
                           na_if(., "") %>%
                           na.omit(.) %>% 
                           sort(.) %>% 
                           paste(., collapse = ", "),
          ) %>%
    ungroup() %>%
    select(- matches("CentreType_")) %>%
    mutate(across(where(is.character), str_squish)) %>%
    distinct %>%


    full_join(school_contact_list_2023 %>%
                  select(- any_of(c("Year", "GaelicUnit", "IntegratedSpecialUnit", "Denomination"))) %>% 
                  mutate(across(CentreStatus, ~ fct_recode(., "Education Authority" = "Local Authority"))) %>%
                  rename_with(~ paste0(., "_fill"), 
                              !any_of(c("LocalAuthority", "SeedCode", "SchoolName", "CentreStatus", "Address", "Postcode")))
             ) %>%

    rowwise() %>%
    mutate(across(matches("CentreType"), ~ str_split(., pattern = "\\s*,\\s*")),
           CentreType = unique(c_across(matches("CentreType"))) %>% 
                           na_if(., "") %>%
                           na.omit(.) %>% 
                           sort(.) %>% 
                           paste(., collapse = ", "),
          ) %>%
    ungroup() %>%
    select(- matches("CentreType_")) %>%
    mutate(across(where(is.character), str_squish)) %>%
    distinct %>%


    full_join(school_rolls_1996_2018 %>%
                  rename_with(~ paste0(., "_fill"), 
                              !any_of(c("LocalAuthority", "SeedCode", "SchoolName", "CentreStatus", "Address", "Postcode")))
             ) %>%

    rowwise() %>%
    mutate(across(matches("CentreType"), ~ str_split(., pattern = "\\s*,\\s*")),
           CentreType = unique(c_across(matches("CentreType"))) %>% 
                           na_if(., "") %>%
                           na.omit(.) %>% 
                           sort(.) %>% 
                           paste(., collapse = ", "),
          ) %>%
    ungroup() %>%
    select(- matches("CentreType_")) %>%
    mutate(across(where(is.character), str_squish)) %>%
    distinct %>%

    arrange(SeedCode) %>%
    glimpse() 

tmp_df %>% 
    group_by(CentreType) %>%
    slice_sample(n = 1)

tmp_df %>%
    slice_head(n = 18)

Joining with `by = join_by(LocalAuthority, SeedCode, SchoolName, CentreStatus,
Address, Postcode)`
Joining with `by = join_by(LocalAuthority, SeedCode, SchoolName, CentreStatus,
Address, Postcode)`
Joining with `by = join_by(LocalAuthority, SeedCode, SchoolName, CentreStatus)`


Rows: 1,769
Columns: 8
$ LocalAuthority <fct> North Lanarkshire, North Lanarkshire, North Lanarkshire…
$ SeedCode       <fct> 1000047, 1000047, 1000047, 1000144, 1000144, 1000144, 1…
$ SchoolName     <fct> "Firpark Primary School", "Firpark Primary School", "Fi…
$ CentreClosed   <lgl> FALSE, FALSE, NA, FALSE, FALSE, NA, FALSE, FALSE, NA, F…
$ CentreStatus   <fct> Education Authority, NA, Local Authority, Education Aut…
$ CentreType     <chr> "PreSchool, Special", "Special", "Special", "Special", …
$ Address        <chr> "Milton Street, Motherwell", NA, NA, "Firpark Street, M…
$ Postcode       <fct> ML1 1DH, NA, NA, ML1 2PR, ML1 2PR, NA, G32 6DJ, NA, NA,…


LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,Address,Postcode
<fct>,<fct>,<fct>,<lgl>,<fct>,<chr>,<chr>,<fct>
Shetland Islands,6103839,Sandwick Junior High School,FALSE,Education Authority,"PreSchool, Primary, Secondary","Sandwick, Shetland",ZE2 9HH
City of Edinburgh,5533236,Leith Academy,FALSE,Education Authority,"PreSchool, Secondary","20 Academy Park, Edinburgh",EH6 8JQ
Grant Aided,8370044,Craighalbert-Scottish Centre for Children with Motor Impairments,NA,Grant Aided,"PreSchool, Special","Craighalbert Centre, 1 Craighalbert Way, Cumbernauld",G68 0LS
City of Edinburgh,5580730,Fettes College,FALSE,Independent,"Primary, Secondary","Carrington Road, Edinburgh",EH4 1QX
East Renfrewshire,8601739,Eastwood High School,FALSE,Education Authority,Secondary,"Capelrig Road, Newton Mearns, East Renfrewshire",G77 6NQ
East Ayrshire,1070339,Eden Park Academy,FALSE,Independent,"Secondary, Special","Pagan Walk, Muirkirk, East Ayrshire",KA18 3PU
North Ayrshire,8280649,Seafield School,NA,Independent,Special,NA,NA


LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,Address,Postcode
<fct>,<fct>,<fct>,<lgl>,<fct>,<chr>,<chr>,<fct>
North Lanarkshire,1000047,Firpark Primary School,FALSE,Education Authority,"PreSchool, Special","Milton Street, Motherwell",ML1 1DH
North Lanarkshire,1000047,Firpark Primary School,FALSE,NA,Special,NA,NA
North Lanarkshire,1000047,Firpark Primary,NA,Local Authority,Special,NA,NA
North Lanarkshire,1000144,Firpark Special School - Secondary,FALSE,Education Authority,Special,"Firpark Street, Motherwell",ML1 2PR
North Lanarkshire,1000144,Firpark Secondary School,FALSE,Education Authority,Special,"Firpark Street, Motherwell",ML1 2PR
North Lanarkshire,1000144,Firpark Secondary,NA,Local Authority,Special,NA,NA
Glasgow City,1000241,Westmuir High School,FALSE,Education Authority,Special,"255 Rigby Street, Glasgow",G32 6DJ
Glasgow City,1000241,Westmuir High School,FALSE,NA,Special,NA,NA
Glasgow City,1000241,Westmuir High School,NA,Local Authority,Special,NA,NA


In [510]:
education_authorities <- centre_postcodes %>%
    filter(!is.na(CentreStatus)) %>% # & (EducationAuthority == "Independent")) %>%
    select(matches("Authority|GeoArea")) %>%
    distinct() %>%
    mutate(across(everything(), as.character)) %>%
    drop_na() %>%
    arrange(GeoArea) %>%
    rename(LocalAuthority = GeoArea) %>%
    
    glimpse()

getEducationAuthority("Fife", education_authorities)
getEducationAuthority("Na h-Eileanan Siar", education_authorities)

Rows: 32
Columns: 2
$ EducationAuthority <chr> "Aberdeen City Council Education Department", "Aber…
$ LocalAuthority     <chr> "Aberdeen City", "Aberdeenshire", "Angus", "Argyll …


[1] "Fife Council Education Department"

[1] "Western Isles Council Education Department"

In [682]:
# all state schools - run a batch fill

tmp_df$EducationAuthority <-
    sapply(tmp_df$LocalAuthority, 
           getEducationAuthority, 
           education_authorities)


n_distinct(tmp_df$EducationAuthority, na.rm = TRUE)
unique(tmp_df$EducationAuthority)

[1] 32

[1] "North Lanarkshire Council Education Department"    
 [2] "City of Glasgow Council Education Department"      
 [3] "Falkirk Council Education Department"              
 [4] "The City of Edinburgh Council Education Department"
 [5] "West Lothian Council Education Department"         
 [6] "South Lanarkshire Council Education Department"    
 [7] "East Dunbartonshire Council Education Department"  
 [8] "Fife Council Education Department"                 
 [9] "Renfrewshire Council Education Department"         
[10] "Aberdeen City Council Education Department"        
[11] "Inverclyde Council Education Department"           
[12] "Scottish Borders Council Education Department"     
[13] "East Ayrshire Council Education Department"        
[14] "Dumfries & Galloway Council Education Department"  
[15] "Perth & Kinross Council Education Department"      
[16] "North Ayrshire Council Education Department"       
[17] "Highland Council Education Department"             
[18] "Aberdeenshire Council Education Department"        
[19] "The Moray Council Education Department"            
[20] "Angus Council Education Department"                
[21] "Dundee City Council Education Department"          
[22] "Midlothian Council Education Department"           
[23] "East Lothian Council Education Department"         
[24] NA                                                  
[25] "Clackmannanshire Council Education Department"     
[26] "Stirling Council Education Department"             
[27] "Orkney Islands Council Education Department"       
[28] "Shetland Islands Council Education Department"     
[29] "Western Isles Council Education Department"        
[30] "Argyll & Bute Council Education Department"        
[31] "South Ayrshire Council Education Department"       
[32] "West Dunbartonshire Council Education Department"  
[33] "East Renfrewshire Council Education Department"

In [683]:
tmp_df %>%
    distinct(LocalAuthority, EducationAuthority, CentreStatus, CentreType) %>%
    filter(if_any(c(CentreStatus, EducationAuthority), ~ (. == "Independent")))

tmp_df %>%
    distinct(LocalAuthority, EducationAuthority, CentreStatus, CentreType) %>%

    mutate(across(EducationAuthority, as.character),
           across(EducationAuthority, ~ case_when((CentreStatus == "Independent") ~ "Independent",
                                                  TRUE ~ .
                                                 )),
           across(EducationAuthority, ~ fct_relevel(., "Independent", after = Inf)),
           ) %>%
    filter(if_any(c(CentreStatus, EducationAuthority), ~ (. == "Independent"))) #%>%

#    glimpse()


LocalAuthority,EducationAuthority,CentreStatus,CentreType
<fct>,<chr>,<fct>,<chr>
City of Edinburgh,The City of Edinburgh Council Education Department,Independent,Special
Glasgow City,City of Glasgow Council Education Department,Independent,"Primary, Secondary"
Fife,Fife Council Education Department,Independent,Special
City of Edinburgh,The City of Edinburgh Council Education Department,Independent,Secondary
Highland,Highland Council Education Department,Independent,Special
Aberdeenshire,Aberdeenshire Council Education Department,Independent,Special
Moray,The Moray Council Education Department,Independent,"Primary, Secondary"
East Ayrshire,East Ayrshire Council Education Department,Independent,"Secondary, Special"
East Ayrshire,East Ayrshire Council Education Department,Independent,Special


LocalAuthority,EducationAuthority,CentreStatus,CentreType
<fct>,<fct>,<fct>,<chr>
City of Edinburgh,Independent,Independent,Special
Glasgow City,Independent,Independent,"Primary, Secondary"
Fife,Independent,Independent,Special
City of Edinburgh,Independent,Independent,Secondary
Highland,Independent,Independent,Special
Aberdeenshire,Independent,Independent,Special
Moray,Independent,Independent,"Primary, Secondary"
East Ayrshire,Independent,Independent,"Secondary, Special"
East Ayrshire,Independent,Independent,Special


In [684]:
dim(centre_postcodes)
dim(tmp_df)

tmp_df <- centre_postcodes %>% 
    # convenience - revert after
    rename_with(~ c("SeedCode", "SchoolName", "LocalAuthority"), c("Centre", "CentreName", "GeoArea")) %>%
    mutate(across(CentreType, ~ str_match(., "(?<=\\w)\\s+(School)$")[, 2], .names = "CentreType_Suffix"),
           across(CentreType, ~ gsub("(?<=\\w)\\s+School$", "", ., perl = TRUE))
          ) %>% 

    full_join(tmp_df %>%
                mutate(across(EducationAuthority, as.character),
                       across(EducationAuthority, ~ case_when((CentreStatus == "Independent") ~ "Independent",
                                                              TRUE ~ .
                                                             )),
                       across(EducationAuthority, ~ fct_relevel(., "Independent", after = Inf))
                       ) %>%
                rename_with(~ paste0(., "_fill"), 
                            !any_of(c("LocalAuthority", "SeedCode", "SchoolName", "CentreStatus", "Address", "Postcode"))) %>%
                mutate(CentreType_Suffix = "School")             
             ) %>%

    mutate(across(EducationAuthority, ~ coalesce(., EducationAuthority_fill)),
           across(EducationAuthority_fill, ~ na_if(., EducationAuthority)),
           #across(matches("EducationAuthority"), ~ na_if(., ""))

           across(CentreClosed, ~ coalesce(., CentreClosed_fill)),
           across(CentreClosed_fill, ~ na_if(., CentreClosed)),
           #across(matches("CentreClosed"), ~ na_if(., ""))

           across(CentreType, ~ coalesce(., CentreType_fill)),
           across(CentreType_fill, ~ na_if(., CentreType_fill)),
          ) %>%
    select_if(~ !all(is.na(.))) %>%

    distinct() %>%
    glimpse()

[1] 987  10

[1] 1769    9

Joining with `by = join_by(SeedCode, SchoolName, CentreStatus, Address, Postcode,
LocalAuthority, CentreType_Suffix)`


Rows: 2,756
Columns: 11
$ SeedCode           <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ SchoolName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <fct> Education Authority, Education Authority, Education…
$ CentreType         <chr> "Special", "Special", "Secondary", "Further Educati…
$ Address            <chr> "Firpark Street,\nMotherwell", "67 Townhead Road,\n…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <fct> North Lanarkshire Council Education Department, Nor…
$ LocalAuthority     <fct> North Lanarkshire, North Lanarkshire, North Lanarks…
$ CentreType_Suffix  <chr> "School", "School", "School", NA, NA, NA, "School",…


In [ ]:
***

In [ ]:
tmp_df <- tmp_df %>%

    group_by(SeedCode) %>%
    mutate(across(SchoolName, ~ if_else((str_to_upper(.) == .), tools::toTitleCase(str_to_lower(.)), .)),
           across(SchoolName, ~ str_c(coalesce(., ""), collapse = "; "), .names = "OtherSchoolNames")) %>%

    rowwise() %>%
    mutate(across(OtherSchoolNames, ~ str_split(., pattern = ";\\s")), 
           OtherSchoolNames = list(str_unique(OtherSchoolNames) %>% 
                                   sort(.)), 
          ) %>% 
    ungroup() %>%
    distinct() %>%

    left_join(centre_postcodes %>%
                  distinct(Centre, CentreName) %>%
                  rename_with(~ c("SeedCode", "SchoolName_"), c("Centre", "CentreName")) %>%
                  mutate(across(matches("SchoolName"), ~ 
                                    if_else((str_to_upper(.) == .), tools::toTitleCase(str_to_lower(.)), .)))
             ) %>%

    rowwise() %>%
    mutate(across(SchoolName, ~ if_else((SeedCode %in% c("5240042")), ., coalesce(SchoolName_, pluck(OtherSchoolNames, 1)))),
           SeedCodeFlag = if_else((SeedCode %in% c("5240042")), "two distinct locations", NA_character_),
           
           OtherSchoolNames = list(OtherSchoolNames[OtherSchoolNames != SchoolName]),
           OtherSchoolNames = str_flatten(OtherSchoolNames, collapse = "; "),
           across(SchoolName_, ~ na_if(., SchoolName)),
          ) %>%
    ungroup() %>%
    distinct() %>% 

#***
    group_by(SeedCode) %>%
    mutate(CentreClosed_fill = as.character(CentreClosed),
           across(c(CentreType, CentreStatus, CentreClosed_fill, EducationAuthority, LocalAuthority), 
                      ~ str_c(coalesce(., ""), collapse = ", ")),
          ) %>%

    rowwise() %>%
    mutate(across(c(CentreType, CentreStatus, LocalAuthority), ~ str_split(., pattern = "\\s*,\\s*")), 
           
           CentreType = list(CentreType[CentreType != ""]),
           CentreType = str_unique(CentreType) %>% 
                           sort(.) %>% 
                           str_flatten(., collapse = ", ", last = " & "),
           
           across(CentreClosed_fill, ~ str_split(., pattern = ",\\s*")), 
           CentreClosed_fill = list(CentreClosed_fill[CentreClosed_fill != ""]),
           CentreClosed_fill = str_unique(CentreClosed_fill) %>% 
                                   sort(.) %>% 
                                   str_flatten(., collapse = ", "),
           across(CentreClosed, ~ if_else(str_detect(CentreClosed_fill, regex("\\bTRUE\\b", ignore_case = TRUE)), 
                                          TRUE, as.logical(CentreClosed_fill))), # value not always set or derivable
           
           
           CentreStatus = list(CentreStatus[CentreStatus != ""]),
           GrantAided = list(CentreStatus[str_detect(CentreStatus, regex("grant", ignore_case = TRUE))]),
           across(CentreStatus, ~ list(CentreStatus[!(CentreStatus %in% GrantAided)])),                  
           across(GrantAided, ~ as.logical(length(.))),
           
           CentreStatus = unique(CentreStatus) %>% 
                                   sort(.) %>% 
                                   str_flatten(., collapse = ", "),
           
           LocalAuthority = list(LocalAuthority[LocalAuthority != ""]),
           GrantAided_2 = list(LocalAuthority[str_detect(LocalAuthority, regex("grant", ignore_case = TRUE))]),
           across(LocalAuthority, ~ list(LocalAuthority[!(LocalAuthority %in% GrantAided_2)])),    
           LocalAuthorityFlag = n_distinct(LocalAuthority) > 1,
           across(GrantAided_2, ~ as.logical(length(.))),
           across(GrantAided, ~ if_else(., ., GrantAided_2)),
           across(GrantAided_2, ~ na_if(., GrantAided)),
           
           LocalAuthority = unique(LocalAuthority) %>% 
                                   sort(.) %>% 
                                   str_flatten(., collapse = ", "),
           across(LocalAuthorityFlag, ~ if_else(., LocalAuthority, NA_character_)),
                   
           across(EducationAuthority, ~ str_split(., pattern = "\\s*,\\s*")), 
           EducationAuthority = list(EducationAuthority[EducationAuthority != ""]),
           EducationAuthority = str_unique(EducationAuthority) %>% 
                                   sort(.) %>% 
                                   str_flatten(., collapse = ", "),
           across(c(CentreStatus, matches("Authority")), ~ na_if(str_squish(.), "")),
          ) %>% 
    ungroup() %>%
    select(- CentreClosed_fill) %>%

# ***
    mutate(across(where(is.character), str_squish)) %>%
    distinct() %>%
    select_if(~ !all(is.na(.))) %>%

    unite("CentreType", matches("^CentreType"), na.rm = TRUE, sep = " ") %>%
    relocate(GrantAided, .after = CentreStatus) %>%
    relocate(LocalAuthorityFlag, .after = LocalAuthority) %>%
    
    glimpse()

In [687]:
tmp_df <- tmp_df %>%

    relocate(CentreClosed, CentreStatus, CentreType, .after = SchoolName) %>%
    relocate(LocalAuthority, .after = EducationAuthority) %>%
    rename_with(~ c("Centre", "GeoArea"), 
                c("SeedCode", "LocalAuthority")) %>%
    rename_with(~ gsub("School", "Centre", .)) %>%

    mutate(across(GeoArea, ~ fct_relevel(., #setdiff(centre_postcodes$GeoArea, summary_stats_schools$LocalAuthority),
                                         "England", "Wales", "Middle East", "Far East", "South Asia", "Other",
                                         after = Inf)),
           
           Country = case_when((GeoArea %in% summary_stats_schools$LocalAuthority) ~ "Scotland",
                               (GeoArea %in% c("England", "Wales")) ~ as.character(GeoArea),
                               TRUE ~ "Other"
                              ),
           across(Country, ~ factor(., levels = c("Scotland", "England", "Wales", "Other")))
          ) %>% 

    glimpse()

    
tmp_df %>%
    group_by(Centre) %>%
    add_tally() %>%
    ungroup() %>%

    filter(n > 1) %>%
    arrange(Centre) %>%
    slice_head(n = 18)

Rows: 2,642
Columns: 15
$ Centre             <fct> 1000144, 1001930, 1002732, 1002759, 1002856, 100295…
$ CentreName         <chr> "Firpark Secondary School", "Buchanan High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <chr> "Education Authority, Local Authority", "Education …
$ CentreType         <chr> "Special School", "Special School", "Secondary Scho…
$ GrantAided         <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Address            <chr> "Firpark Street, Motherwell", "67 Townhead Road, Co…
$ Postcode           <fct> ML1 2PR, ML5 2HT, G67 2UF, KA13 6DE, DD3 8LE, EH11 …
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <chr> "North Lanarkshire Council Education Department", "…
$ GeoArea            <fct> "North Lanarkshire", "North Lanarkshire", "North La…
$ LocalAuthorityFlag <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ OtherCentreNam

Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,LocalAuthorityFlag,OtherCentreNames,SeedCodeFlag,Country,n
<fct>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<fct>,<lgl>,<chr>,<fct>,<chr>,<chr>,<chr>,<fct>,<int>
1000144,Firpark Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,"Firpark Street, Motherwell",ML1 2PR,FALSE,North Lanarkshire Council Education Department,North Lanarkshire,NA,Firpark Secondary; Firpark Special School - Secondary,NA,Scotland,3
1000144,Firpark Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,"Firpark Street, Motherwell",ML1 2PR,NA,North Lanarkshire Council Education Department,North Lanarkshire,NA,Firpark Secondary; Firpark Special School - Secondary,NA,Scotland,3
1000144,Firpark Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,NA,North Lanarkshire Council Education Department,North Lanarkshire,NA,Firpark Secondary; Firpark Special School - Secondary,NA,Scotland,3
1001930,Buchanan High School,FALSE,"Education Authority, Local Authority",Special School,FALSE,"67 Townhead Road, Coatbridge",ML5 2HT,FALSE,North Lanarkshire Council Education Department,North Lanarkshire,NA,Buchanan High,NA,Scotland,3
1001930,Buchanan High School,FALSE,"Education Authority, Local Authority",Special School,FALSE,"67 Townhead Road, Coatbridge",ML5 2HT,NA,North Lanarkshire Council Education Department,North Lanarkshire,NA,Buchanan High,NA,Scotland,3
1001930,Buchanan High School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,NA,North Lanarkshire Council Education Department,North Lanarkshire,NA,Buchanan High,NA,Scotland,3
1002732,Cumbernauld Academy,FALSE,"Education Authority, Local Authority",Secondary School,FALSE,"Kildrum Ring Road, Kildrum, Cumbernauld",G67 2UF,FALSE,North Lanarkshire Council Education Department,North Lanarkshire,NA,,NA,Scotland,4
1002732,Cumbernauld Academy,FALSE,"Education Authority, Local Authority",Secondary School,FALSE,"South Kildrum Ring Road, Cumbernauld",G67 2UF,NA,North Lanarkshire Council Education Department,North Lanarkshire,NA,,NA,Scotland,4
1002732,Cumbernauld Academy,FALSE,"Education Authority, Local Authority",Secondary School,FALSE,"Kildrum Ring Road, Kildrum, Cumbernauld",G67 2UF,NA,North Lanarkshire Council Education Department,North Lanarkshire,NA,,NA,Scotland,4


In [550]:
*** need to finish address cleaning

summary_stats_schools_corrections <- read_xlsx("sta_it_402/data/summary_stats_schools_corrections.xlsx", 
                                                    trim_ws = TRUE, col_types = "text") %>% 
    filter(rowSums(is.na(.)) != ncol(.)) %>% 
    select(SeedCode, matches("LocalAuthority|Address|Postcode")) %>%
    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%
    mutate(across(matches("^Correction"), ~ coalesce(., get(str_remove(cur_column(), "Correction")))),
            
           across(Address, ~ str_remove(., paste0("[[:punct:][:blank:]]*", as.character(LocalAuthority), "$"))),
          ) %>%
    distinct(across(!c(LocalAuthority, Address, Postcode))) %>%
    
    glimpse()


n_distinct(summary_stats_schools_corrections$SeedCode)

summary_stats_schools_corrections %>%
    filter(if_any(everything(), is.na))

New names:
• `` -> `...6`
• `` -> `...9`


Rows: 352
Columns: 3
$ SeedCode           <chr> "5747147", "1003038", "1003658", "1070339", "510213…
$ CorrectionAddress  <chr> "Oxgang Road, Grangemouth", "Hopefield Road, Blackb…
$ CorrectionPostcode <chr> "FK3 9HP", "EH47 7HZ", "PA3 3DP", "KA18 3PU", "KW1 …


[1] 284

SeedCode,CorrectionAddress,CorrectionPostcode
<chr>,<chr>,<chr>


In [688]:
dim(tmp_df)

tmp_df <- tmp_df %>%
    
    mutate(across(Address, ~ str_remove(., paste0("[[:punct:][:blank:]]*", as.character(GeoArea), "$")))) %>%
    distinct() %>%

    left_join(summary_stats_schools_corrections, by = c("Centre" = "SeedCode")) %>%
              
    filter(!(Centre %in% c("8442746", "5720540", "5700248", "8441049", "8240140", "1000047", "8440344", "8440441", 
                           "8441146", "8441448", "8443440", "8447241", "8459045", "8459142", "1001922", "5245044",
                           "5702941", "5726840", "8242143", "8315248", "5232147", "1004840", "1004859")
                          ))) %>% # primaries (mainly special)

    mutate(across(where(is.character), str_squish),
           
           across(CentreName, ~ case_when((Centre == "8370044") ~ "Craighalbert-Scottish Centre for Children with Motor Impairments",
                                          (str_to_upper(.) == .) ~ tools::toTitleCase(str_to_lower(.)),
                                          TRUE ~ .
                                         )),
           
           across(matches("Correction"), ~ na_if(., "")),
           FlagCorrections = case_when(if_all(matches("Correction"), ~ !is.na(.) & 
                                              (. != get(str_remove(cur_column(), "Correction")))) ~ "Address_Postcode",
                                       (Address != CorrectionAddress) ~ "Address",
                                       (Postcode != CorrectionPostcode) ~ "Postcode",
                                      ),
           across(matches("^Correction"), ~ coalesce(., get(str_remove(cur_column(), "Correction")))),
           
           across(AddressFlag, ~ if_else(is.na(.), FALSE, as.logical(.))),
          ) %>%
    select_if(~ !all(is.na(.) & (. != FALSE))) %>%
    select(- c(Address, Postcode)) %>%

    distinct() %>% 
    arrange(Centre) %>%
    rename_with(~ str_remove(., "Correction"), matches("^Correction")) %>%


    pivot_wider(names_from = FlagCorrections, names_prefix = "FlagCorrections_", values_from = FlagCorrections) %>%
    separate("FlagCorrections_Address_Postcode", c("FlagCorrections_Address_1", "FlagCorrections_Postcode_1")) %>% 

    rowwise() %>%
    mutate(FlagCorrections = unique(c_across(matches("FlagCorrections_"))) %>% 
                               na.omit(.) %>% 
                               sort(.) %>% 
                               paste(., collapse = ", "),
          ) %>%
    ungroup() %>%
    select(- matches("FlagCorrections_")) %>%
    relocate(Address, Postcode, .before = AddressFlag) %>%
    relocate(Country, .after = GeoArea) %>%

    glimpse()

tmp_df %>%
    group_by(CentreType, FlagCorrections) %>%
    slice_sample(n = 1)

[1] 2642   15

Warning message in left_join(., summary_stats_schools_corrections, by = c(Centre = "SeedCode")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 557 of `x` matches multiple rows in `y`.
ℹ Row 85 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"`
  to silence this warning.”


Rows: 1,670
Columns: 16
$ Centre             <chr> "1000144", "1000144", "1000241", "1001930", "100193…
$ CentreName         <chr> "Firpark Secondary School", "Firpark Secondary Scho…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <chr> "Education Authority, Local Authority", "Education …
$ CentreType         <chr> "Special School", "Special School", "Special School…
$ GrantAided         <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Address            <chr> "Firpark Street, Motherwell", NA, "255 Rigby Street…
$ Postcode           <chr> "ML1 2PR", NA, "G32 6DJ", "ML5 2HT", NA, "G67 2UF",…
$ AddressFlag        <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ EducationAuthority <chr> "North Lanarkshire Council Education Department", "…
$ GeoArea            <fct> "North Lanarkshire", "North Lanarkshire", "Glasgow …
$ Country            <fct> Scotland, Scotland, Scotland, Scotland, Scotland, S…
$ LocalAuthority

Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,SeedCodeFlag,FlagCorrections
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
5995530,Community Learning and Development - Nithsdale,FALSE,Education Authority,Further Education Centre,FALSE,"Community & Customer Services, Dumfries & Galloway Council, North West Resource Centre, College Drive Lincluden DUMFRIES",DG2 0BX,TRUE,Dumfries & Galloway Council Education Department,Dumfries & Galloway,Scotland,NA,,NA,
1002953,Edinburgh College,FALSE,NA,Further Education College,FALSE,"Bankhead Avenue, Edinburgh",EH11 4DE,FALSE,NA,City of Edinburgh,Scotland,NA,,NA,
3012360,West Lothian Training Centre,FALSE,NA,HM Prisons,FALSE,"HM Prison Addiewell, 9 Station Road, Addiewell, West Calder",EH55 8QF,FALSE,NA,West Lothian,Scotland,NA,,NA,
3010120,School of International Education Guizhou Normal University,FALSE,NA,Higher Education Institutions,FALSE,"School of Foreign Languages, 116 Baoshan North Road, Guiyang, Guizhou Province",550001,FALSE,NA,Far East,Other,NA,,NA,
5533236,Leith Academy,FALSE,"Education Authority, Local Authority",PreSchool & Secondary School,FALSE,NA,NA,FALSE,The City of Edinburgh Council Education Department,City of Edinburgh,Scotland,NA,,NA,
5110432,Farr High School,FALSE,"Education Authority, Local Authority",PreSchool & Secondary School,FALSE,"Bettyhill, By Thurso",KW14 7SS,FALSE,Highland Council Education Department,Highland,Scotland,NA,Farr High,NA,"Address, Postcode"
5331749,Kingspark School,FALSE,"Education Authority, Local Authority",PreSchool & Special School,FALSE,"5 Glenaffric Terrace, Dundee",DD3 8HF,FALSE,Dundee City Council Education Department,Dundee City,Scotland,NA,Kingspark,NA,
8570140,Stanmore House School,FALSE,Independent,PreSchool & Special School,TRUE,"Capability Scotland Grant Aided School, Lanark",ML11 7RR,FALSE,Independent,South Lanarkshire,Scotland,NA,,NA,Address
5447046,Calaiswood School,FALSE,"Education Authority, Local Authority",PreSchool & Special School,FALSE,"Nightingale Place, Dunfermline",KY11 8LW,FALSE,Fife Council Education Department,Fife,Scotland,NA,Robert Henryson School,NA,"Address, Postcode"


In [697]:

summary_stats_schools_corrections %>%
    filter(SeedCode == "5533236")

tmp_df %>%
    filter(if_any(c(Address, Postcode), is.na)) %>%
    nrow()

tmp_df %>%
    filter(if_any(c(Address, Postcode), is.na)) %>%
    anti_join(summary_stats_schools_corrections, by = c("Centre" = "SeedCode"))

SeedCode,CorrectionAddress,CorrectionPostcode
<chr>,<chr>,<chr>


[1] 492

Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Address,Postcode,AddressFlag,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,SeedCodeFlag,FlagCorrections
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>
1000144,Firpark Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,North Lanarkshire Council Education Department,North Lanarkshire,Scotland,NA,Firpark Secondary; Firpark Special School - Secondary,NA,
1001930,Buchanan High School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,North Lanarkshire Council Education Department,North Lanarkshire,Scotland,NA,Buchanan High,NA,
1002945,Panmure St Ann's Special School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,The City of Edinburgh Council Education Department,City of Edinburgh,Scotland,NA,,NA,
1003046,Kear Campus School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,South Lanarkshire Council Education Department,South Lanarkshire,Scotland,NA,Kear Campus,NA,
1003135,Ashton Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,City of Glasgow Council Education Department,Glasgow City,Scotland,NA,,NA,
1004174,Lomond View Academy,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,Inverclyde Council Education Department,Inverclyde,Scotland,NA,Lomond View Academy (Special),NA,
1004182,Craigmarloch School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,NA,FALSE,Inverclyde Council Education Department,Inverclyde,Scotland,NA,,NA,
3002349,Centre of Excellence for Technical and Vocational Education,FALSE,NA,Higher Education Institutions,FALSE,"Sh. Khalifa bin Salman Institute, Ministry of Education, Muharraq, PO Box 22909",NA,TRUE,NA,Middle East,Other,NA,,NA,
3003647,ADNOC SQA Centre,FALSE,NA,Public Sector Employer,FALSE,"Ms. Reem A Mohamed, SQA CENTRE, PEOPLE DEVELOPMENT FUNCTIONS, 24th Floor ADNOC HQ Corniche Street, PO Box 898 Abu Dhabi",NA,FALSE,NA,Middle East,Other,NA,,NA,


In [698]:
***
tmp_df_bkp <- tmp_df

In [699]:
# interim - postcodes only
dim(tmp_df)

tmp_df %>%
    select(- matches("Address|Corrections")) %>%
    distinct() %>%
    glimpse()

tmp_df %>%
    select(- matches("Address|Corrections")) %>%
    distinct() %>%
    group_by(Centre) %>%
    add_tally() %>%
    ungroup() %>%

    filter(n > 1) %>%
    arrange(Centre) %>%
    nrow()


tmp_df <- tmp_df %>%
    select(- matches("Address|Corrections")) %>%
    distinct()

# END - interim - postcodes only

[1] 1670   16

Rows: 1,583
Columns: 13
$ Centre             <chr> "1000144", "1000144", "1000241", "1001930", "100193…
$ CentreName         <chr> "Firpark Secondary School", "Firpark Secondary Scho…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <chr> "Education Authority, Local Authority", "Education …
$ CentreType         <chr> "Special School", "Special School", "Special School…
$ GrantAided         <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Postcode           <chr> "ML1 2PR", NA, "G32 6DJ", "ML5 2HT", NA, "G67 2UF",…
$ EducationAuthority <chr> "North Lanarkshire Council Education Department", "…
$ GeoArea            <fct> "North Lanarkshire", "North Lanarkshire", "Glasgow …
$ Country            <fct> Scotland, Scotland, Scotland, Scotland, Scotland, S…
$ LocalAuthorityFlag <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ OtherCentreNames   <chr> "Firpark Secondary; Firpark Special School - Second…
$ SeedCodeFlag  

[1] 518

In [650]:
summary_stats_schools %>%
    filter(SeedCode == "5240042")

school_contact_list_2023 %>%
    filter(SeedCode == "5240042")

school_rolls_1996_2018 %>%
    #filter(str_detect(CentreName, regex("global|focus", ignore_case = TRUE)))
    filter(SeedCode == "5240042")

tmp_df %>%
    filter(Centre == "5240042")

Year,LocalAuthority,SeedCode,SchoolName,CentreClosed,CentreStatus,CentreType,GrantAided,Address,Postcode
<ord>,<fct>,<fct>,<fct>,<lgl>,<fct>,<fct>,<lgl>,<chr>,<fct>
2017,Aberdeen City,5240042,Pupil Support Service,FALSE,Education Authority,Special,FALSE,"St Nicholas Pupil Support Centre, Balgownie Road, Bridge of Don Aberdeen",AB22 8JS
2018,Aberdeen City,5240042,Pupil Support Service,FALSE,Education Authority,Special,FALSE,"St Nicholas Pupil Support Centre, Balgownie Road, Bridge of Don Aberdeen",AB22 8JS
2019,Aberdeen City,5240042,Virtual School,FALSE,Education Authority,Special,FALSE,"Marischal College, Broad Street, Aberdeen",AB10 1AB


Year,SeedCode,LocalAuthority,SchoolName,CentreStatus,CentreType,GaelicUnit,IntegratedSpecialUnit,Denomination,Address,Postcode
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<lgl>,<lgl>,<fct>,<chr>,<fct>
2023,5240042,Aberdeen City,Virtual School,Local Authority,Special,FALSE,FALSE,Non-denominational,"Marischal College, Broad Street, Aberdeen",AB10 1AB


LocalAuthority,SeedCode,SchoolName,CentreType,CentreStatus
<fct>,<fct>,<fct>,<fct>,<fct>
Aberdeen City,5240042,Pupil Support Service (on Site Secondary),Special,Local Authority


Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Postcode,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,CentreFlag
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>
5240042,Pupil Support Service,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB22 8JS,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service (on Site Secondary); Virtual School,two distinct locations
5240042,Pupil Support Service,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB10 1AB,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service (on Site Secondary); Virtual School,two distinct locations
5240042,Virtual School,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB22 8JS,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service; Pupil Support Service (on Site Secondary),two distinct locations
5240042,Virtual School,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB10 1AB,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service; Pupil Support Service (on Site Secondary),two distinct locations
5240042,Pupil Support Service (on Site Secondary),FALSE,"Education Authority, Local Authority",Special School,FALSE,AB22 8JS,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service; Virtual School,two distinct locations
5240042,Pupil Support Service (on Site Secondary),FALSE,"Education Authority, Local Authority",Special School,FALSE,AB10 1AB,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service; Virtual School,two distinct locations


In [680]:
dim(tmp_df)
n_distinct(tmp_df$Centre)

tmp_df %>%
    group_by(Centre) %>%
    add_tally() %>%
    ungroup() %>%

    filter(n > 1) %>%
    arrange(Centre) %>%
    #slice_head(n = 18) %>%
    
#    distinct(across(matches("Centre")), Postcode) %>%

#    mutate(test = (Centre %in% summary_stats_schools_corrections$SeedCode)) %>%

    arrange(Centre, CentreName) #%>%desc(test), 
#    filter(!test) %>%
#
#    left_join(tmp_df_bkp %>%
#                  distinct(GeoArea, Centre, CentreName, Address, Postcode)
#             ) %>%
#
#    select(GeoArea, Centre, CentreName, Address, Postcode) %>%
#    rename_with(~ c("LocalAuthority", "SeedCode", "SchoolName"), c("GeoArea", "Centre", "CentreName")) %>%
#    relocate(LocalAuthority) %>%
#    write_csv("sta_it_402/data/summary_stats_schools.csv", na = "")

[1] 1583   13

[1] 1322

Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Postcode,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,CentreFlag,n
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>,<int>
1000144,Firpark Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,ML1 2PR,North Lanarkshire Council Education Department,North Lanarkshire,Scotland,NA,Firpark Secondary; Firpark Special School - Secondary,NA,2
1000144,Firpark Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,North Lanarkshire Council Education Department,North Lanarkshire,Scotland,NA,Firpark Secondary; Firpark Special School - Secondary,NA,2
1001930,Buchanan High School,FALSE,"Education Authority, Local Authority",Special School,FALSE,ML5 2HT,North Lanarkshire Council Education Department,North Lanarkshire,Scotland,NA,Buchanan High,NA,2
1001930,Buchanan High School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,North Lanarkshire Council Education Department,North Lanarkshire,Scotland,NA,Buchanan High,NA,2
1002945,Panmure St Ann's Special School,FALSE,"Education Authority, Local Authority",Special School,FALSE,EH1 1TQ,The City of Edinburgh Council Education Department,City of Edinburgh,Scotland,NA,,NA,2
1002945,Panmure St Ann's Special School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,The City of Edinburgh Council Education Department,City of Edinburgh,Scotland,NA,,NA,2
1003046,Kear Campus School,FALSE,"Education Authority, Local Authority",Special School,FALSE,G72 9UJ,South Lanarkshire Council Education Department,South Lanarkshire,Scotland,NA,Kear Campus,NA,2
1003046,Kear Campus School,FALSE,"Education Authority, Local Authority",Special School,FALSE,NA,South Lanarkshire Council Education Department,South Lanarkshire,Scotland,NA,Kear Campus,NA,2
1003135,Ashton Secondary School,FALSE,"Education Authority, Local Authority",Special School,FALSE,G33 3SW,City of Glasgow Council Education Department,Glasgow City,Scotland,NA,,NA,2


In [764]:
# fill in remaining (addresses) and postcoedes + manual correction
dim(tmp_df)
n_distinct(tmp_df$Centre)

tmp_df <- tmp_df %>%
    filter(Centre != "5240042") %>%
    group_by(Centre) %>%
    mutate(across(Postcode, ~ str_c(coalesce(., ""), collapse = ", "))) %>%

    rowwise() %>%
    mutate(across(Postcode, ~ str_split(., pattern = ",\\s*")),
           Postcode = list(Postcode[Postcode != ""]),
           Postcode = str_unique(Postcode) %>%
                           sort(.) %>%
                           str_flatten(., collapse = ", ", last = " & "),
           ) %>%
    ungroup() %>%

bind_rows(tmp_df %>%
            filter(Centre == "5240042") %>%
            mutate(across(OtherCentreNames, ~ case_when((Centre == "5240042") ~ paste(., CentreName, sep = "; "),
                                                        TRUE ~ .
                                                 )),
                   across(CentreName, ~ case_when((Centre == "5240042") ~
                                                      case_when(str_detect(CentreName, regex("Pupil Support Service", ignore_case = TRUE)) ~ "Pupil Support Service",
                                                                #str_detect(CentreName, regex("Virtual School", ignore_case = TRUE)) ~ .,
                                                                TRUE ~ .
                                                               ),
                                                  TRUE ~ .
                                                 )),
                   across(Postcode, ~ case_when((Centre == "5240042") ~
                                                      case_when(str_detect(CentreName, regex("Pupil Support Service", ignore_case = TRUE)) ~ "AB22 8JS",
                                                                str_detect(CentreName, regex("Virtual School", ignore_case = TRUE)) ~ "AB10 1AB",
                                                                TRUE ~ .
                                                               ),
                                                  TRUE ~ .
                                                 )),
                  ) %>%

            rowwise() %>%
            mutate(across(OtherCentreNames, ~ str_split(., pattern = ";\\s")), 
                   OtherCentreNames = list(str_unique(OtherCentreNames) %>% 
                                           sort(.)), 
                  ) %>% 

            mutate(OtherCentreNames = list(OtherCentreNames[OtherCentreNames != CentreName]),
                   OtherCentreNames = str_flatten(OtherCentreNames, collapse = "; "),
                  ) %>%
            ungroup()
    
    ) %>%
    distinct() 


tmp_df %>%
    filter(Centre == "5240042")

[1] 1583   13

[1] 1322

Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Postcode,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,SeedCodeFlag
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>
5240042,Pupil Support Service,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB22 8JS,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service (on Site Secondary); Virtual School,two distinct locations
5240042,Virtual School,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB10 1AB,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service; Pupil Support Service (on Site Secondary),two distinct locations


In [780]:

tmp_df %>%
    filter(Centre %in% c("5240042", "1002848", "5720540", "5720648"))

Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Postcode,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,SeedCodeFlag
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>
1002848,Inclusion & Wellbeing Service,FALSE,"Education Authority, Local Authority",Special School,FALSE,FK2 9PB,Falkirk Council Education Department,Falkirk,Scotland,NA,Inclusion and Wellbeing Service; Mariner Support Service; Mariner Support Services,NA
5720648,Secondary Student Support Service,FALSE,"Education Authority, Local Authority",Special School,FALSE,FK7 0LL,Stirling Council Education Department,Stirling,Scotland,NA,Stirling Inclusion Support Service (Secondary),NA
5240042,Pupil Support Service,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB22 8JS,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service (on Site Secondary); Virtual School,two distinct locations
5240042,Virtual School,FALSE,"Education Authority, Local Authority",Special School,FALSE,AB10 1AB,Aberdeen City Council Education Department,Aberdeen City,Scotland,NA,Pupil Support Service; Pupil Support Service (on Site Secondary),two distinct locations


In [842]:
schools_closed_up_to_2023 <- read_xlsx(paste(path_to_file_store, "demographic_data", "school_contact_list_30_April_2023.xlsx", sep = "/"),
          trim_ws = TRUE, 
          sheet = "Schools Opened or closed", col_types = "text",
          skip = 6, 
          na = c("*", "#", ".", "..", "z", "x", "N/A"), 
         ) %>% 
    filter(rowSums(is.na(.)) != ncol(.)) %>% 
    select(matches("Status|Seed|name|Sector|Date")) %>% 
 
    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>%
    mutate(across(everything(), str_squish),
           across(matches("SchoolName"), ~ if_else((str_to_upper(.) == .), str_to_lower(.), .)),
           across(matches("SchoolName"), tools::toTitleCase),
           
           across(matches("Date"), ~ as.Date(as.integer(.), origin = excel_date_origin)),
          ) %>%
    filter((Sector != "Primary") & (Status != "Opened")) %>%
    glimpse() 

unique(schools_closed_up_to_2023$Status)
unique(schools_closed_up_to_2023$Sector)

schools_closed_up_to_2023 %>%
    group_by(Status) %>%
    slice_sample(n = 2)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `across(matches("Date"), ~as.Date(as.integer(.), origin =
  excel_date_origin))`.
Caused by warning in `as.Date()`:
! NAs introduced by coercion”


Rows: 133
Columns: 7
$ Status         <chr> "Closed", "Closed", "Closed", "Closed", "Closed", "Clos…
$ SeedCode       <chr> "5235731", "5240140", "5245745", "5210046", "5300142", …
$ SchoolName     <chr> "Bankhead Academy", "Pupil Support Service (South)", "R…
$ Sector         <chr> "Secondary", "Special", "Special", "Special", "Special"…
$ DateOpened     <date> 1970-08-31, 2003-04-22, 1974-08-31, 2006-03-01, 2006-0…
$ DateClosed     <date> 2009-10-25, 2010-07-02, 2011-07-01, 2011-09-13, 2007-0…
$ DateMothballed <date> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA…


[1] "Closed"     "Mothballed" "Split"

[1] "Secondary" "Special"

Status,SeedCode,SchoolName,Sector,DateOpened,DateClosed,DateMothballed
<chr>,<chr>,<chr>,<chr>,<date>,<date>,<date>
Closed,5244838,Torry Academy,Secondary,1970-08-31,2018-06-06,NA
Closed,8441944,Richmond Park School,Special,1975-08-31,2010-06-25,NA
Mothballed,8448744,Ladywell School,Special,NA,NA,2014-07-31
Mothballed,8448744,Ladywell School,Special,NA,NA,2014-07-31
Split,1000144,Firpark Special School - Secondary,Special,2011-08-17,NA,NA
Split,1000047,Firpark Special School - Primary,Special,2011-08-15,NA,NA


In [861]:
schools_renamed_up_to_2023 <- read_xlsx(paste(path_to_file_store, "demographic_data", "school_contact_list_30_April_2023.xlsx", sep = "/"),
          trim_ws = TRUE, 
          sheet = "Schools renamed", col_types = "text",
          skip = 6, 
          na = c("*", "#", ".", "..", "z", "x", "N/A"), 
         ) %>% 
    filter(rowSums(is.na(.)) != ncol(.)) %>% 
    distinct() %>% # needed ...
 
    rename_with(snakecase::to_upper_camel_case, abbreviations = abbreviations) %>% 
    select(- c(LocalAuthority, OtherChanges)) %>%
    rename(Year = YearChanged) %>%

    group_by(SeedCode) %>%


    mutate(across(everything(), str_squish),
           across(matches("SchoolName"), ~ if_else((str_to_upper(.) == .), str_to_lower(.), .)),
           across(matches("SchoolName"), tools::toTitleCase),
           
           across(Year, ~ coalesce(as.integer(.), as.integer(-1))),
           SchoolName = NewSchoolName[(Year == max(Year, na.rm = TRUE))],# which.min((Year)),
           across(Year, ~ na_if(., -1)),
           across(matches("\\w+SchoolName"), ~ str_c(coalesce(., ""), collapse = "; ")),
          ) %>%

    rowwise() %>%
    mutate(PreviousSchoolNames = str_c(across(matches("\\w+SchoolName")), collapse = "; "),
           across(PreviousSchoolNames, ~ str_split(., pattern = ";\\s")),
           PreviousSchoolNames = list(PreviousSchoolNames[!(PreviousSchoolNames %in% c("", SchoolName))]),
           
           PreviousSchoolNames = str_unique(PreviousSchoolNames) %>%
                                       sort(.) %>% 
                                       str_flatten(., collapse = "; "),
          ) %>%

    ungroup() %>%
    relocate(Year) %>%
    select(- matches("\\w+SchoolName$")) %>%

    glimpse()

Rows: 145
Columns: 4
$ Year                <int> 1999, 2001, 2003, 2003, 2005, 2005, 2005, 2006, 20…
$ SeedCode            <chr> "5408849", "5427347", "8338841", "5409349", "84119…
$ SchoolName          <chr> "Glenrothes Education Centre", "Cupar Primary Educ…
$ PreviousSchoolNames <chr> "Rimbleton House School", "Kilmarton Primary Educa…


In [862]:
glimpse(tmp_df)

Rows: 1,323
Columns: 13
$ Centre             <chr> "1000144", "1000241", "1001930", "1002732", "100275…
$ CentreName         <chr> "Firpark Secondary School", "Westmuir High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <chr> "Education Authority, Local Authority", "Education …
$ CentreType         <chr> "Special School", "Special School", "Special School…
$ GrantAided         <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Postcode           <chr> "ML1 2PR", "G32 6DJ", "ML5 2HT", "G67 2UF", "KA13 6…
$ EducationAuthority <chr> "North Lanarkshire Council Education Department", "…
$ GeoArea            <fct> "North Lanarkshire", "Glasgow City", "North Lanarks…
$ Country            <fct> Scotland, Scotland, Scotland, Scotland, Scotland, S…
$ LocalAuthorityFlag <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ OtherCentreNames   <chr> "Firpark Secondary; Firpark Special School - Second…
$ SeedCodeFlag  

In [874]:
centre_postcodes %>%
    filter(Centre %in% c("1004840", "1004859"))

tmp_df <- tmp_df %>%
    
              
    filter(!(Centre %in% c("8442746", "5720540", "5700248", "8441049", "8240140", "1000047", "8440344", "8440441", 
                           "8441146", "8441448", "8443440", "8447241", "8459045", "8459142", "1001922", "5245044",
                           "5702941", "5726840", "8242143", "8315248", "5232147", "1004840", "1004859")
                          )) %>%
    glimpse()



Centre,CentreName,CentreClosed,CentreStatus,CentreType,Address,Postcode,AddressFlag,EducationAuthority,GeoArea
<fct>,<chr>,<lgl>,<fct>,<fct>,<chr>,<fct>,<lgl>,<fct>,<fct>


Rows: 1,321
Columns: 13
$ Centre             <chr> "1000144", "1000241", "1001930", "1002732", "100275…
$ CentreName         <chr> "Firpark Secondary School", "Westmuir High School",…
$ CentreClosed       <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ CentreStatus       <chr> "Education Authority, Local Authority", "Education …
$ CentreType         <chr> "Special School", "Special School", "Special School…
$ GrantAided         <lgl> FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FALSE, FA…
$ Postcode           <chr> "ML1 2PR", "G32 6DJ", "ML5 2HT", "G67 2UF", "KA13 6…
$ EducationAuthority <chr> "North Lanarkshire Council Education Department", "…
$ GeoArea            <fct> "North Lanarkshire", "Glasgow City", "North Lanarks…
$ Country            <fct> Scotland, Scotland, Scotland, Scotland, Scotland, S…
$ LocalAuthorityFlag <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ OtherCentreNames   <chr> "Firpark Secondary; Firpark Special School - Second…
$ SeedCodeFlag  

In [940]:
tmp_df %>%

# *** CHANGE TO LEFT_JOIN ***
inner_join(schools_renamed_up_to_2023 %>%
                 select(- Year),
              by = c("Centre" = "SeedCode")
             ) %>%

# ***
bind_rows(tmp_df %>%
          distinct() %>%
          filter((Centre %in% c("5240042", "5281040", "5281334", "5570948", "5571146", "8470332", "8471045", 
                                 "8570140", "8670242", "8680140")) | 
                 if_any(matches("Authority"), ~ str_detect(., regex("grant", ignore_case = TRUE))))
         ) %>%
arrange(Centre, CentreName) %>%
    
    distinct(Centre, across(matches("School|Centre"))) %>% # |Authority")), EducationAuthority, Postcode
# ***
    distinct() %>%
    

    rowwise() %>%
    mutate(across(matches("\\w+Name"), ~ coalesce(., "")),
           across(OtherCentreNames, ~ str_c(across(matches("\\w+Name")), collapse = "; ")),            
          ) %>%

    group_by(Centre) %>%
    mutate(across(OtherCentreNames, ~ str_c(., collapse = "; ")),
           across(SchoolName, ~ str_c(., collapse = "; ")),
          ) %>%

    rowwise() %>%
    mutate(across(SchoolName, ~ str_split(., pattern = ";\\s")),
           SchoolName = list(SchoolName[SchoolName != ""]),
           SchoolName = str_unique(SchoolName) %>%
                           sort(.) %>% 
                           str_flatten(., collapse = "; "),
           across(SchoolName, ~ na_if(., "")),
           
           across(CentreName, ~ coalesce(SchoolName, .)),
          
           across(OtherCentreNames, ~ str_split(., pattern = ";\\s")),
           OtherCentreNames = list(OtherCentreNames[!(OtherCentreNames %in% c("", SchoolName))]),
           
           OtherCentreNames = str_unique(OtherCentreNames) %>%
                                       sort(.) %>% 
                                       str_flatten(., collapse = "; "),
          ) %>%

    ungroup() %>%
    select(- matches("SchoolName")) %>%

# end test

    distinct()

Warning message in left_join(., schools_renamed_up_to_2023 %>% select(-Year), by = c(Centre = "SeedCode")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 39 of `x` matches multiple rows in `y`.
ℹ Row 98 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"`
  to silence this warning.”


Centre,CentreName,CentreClosed,CentreStatus,CentreType,GrantAided,Postcode,EducationAuthority,GeoArea,Country,LocalAuthorityFlag,OtherCentreNames,SeedCodeFlag
<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<fct>,<fct>,<chr>,<chr>,<chr>
5281040,Camphill School Aberdeen,FALSE,Independent,Special School,FALSE,AB15 9EP,Independent,"Aberdeen City, Aberdeenshire",Other,"Aberdeen City, Aberdeenshire",Camphill Rudolf Steiner Schools; Camphill School Aberdeen,NA
5281334,Lathallan School,FALSE,Independent,Primary & Secondary School,FALSE,DD10 0HN,Independent,"Aberdeenshire, Angus",Other,"Aberdeenshire, Angus",Lathallan Preparatory School,NA
5570948,The Donaldson Trust,FALSE,Independent,PreSchool & Special School,TRUE,EH49 6HZ,Independent,West Lothian,Scotland,NA,Donaldson's College; Donaldson's School; The Donaldson Trust,NA
5571146,Royal Blind School,FALSE,Independent,PreSchool & Special School,TRUE,EH10 4SG,Independent,City of Edinburgh,Scotland,NA,Royal Blind School,NA
8470332,Jordanhill School,FALSE,Independent,Primary & Secondary School,TRUE,G13 1SP,Independent,Glasgow City,Scotland,NA,Jordanhill; Jordanhill School,NA
8471045,East Park School,FALSE,Independent,Special School,TRUE,G20 9TD,Independent,Glasgow City,Scotland,NA,East Park School,NA
8570140,Stanmore House School,FALSE,Independent,PreSchool & Special School,TRUE,ML11 7RR,Independent,South Lanarkshire,Scotland,NA,Stanmore House School,NA
8670242,Corseford School,FALSE,Independent,Special School,TRUE,PA10 2NT,Independent,Renfrewshire,Scotland,NA,Corseford Residential School,NA
8680140,Ardfern,FALSE,Independent,Special School,FALSE,G83 0TL,Independent,"Renfrewshire, West Dunbartonshire",Other,"Renfrewshire, West Dunbartonshire",Ardfern; Ardfern Learning Centre - Johnstone,NA


In [ ]:
***

In [ ]:
# start test

tmp_df %>%
    distinct() %>%
    group_by(Centre) %>%
    add_tally() %>%
    ungroup() %>%

    filter(n > 1) %>%
    arrange(Centre) %>%
    slice_head(n = 18) %>%

bind_rows(tmp_df %>%
          distinct() %>%
          filter((Centre %in% c("5240042", "5281040", "5281334", "5570948", "5571146", "8470332", "8471045", 
                                 "8570140", "8670242", "8680140")) | 
                 if_any(matches("Authority"), ~ str_detect(., regex("grant", ignore_case = TRUE))))
         )

In [ ]:
#start test

tmp_df %>%
    #mutate(across(Address, tools::toTitleCase)) %>%

    mutate(across(Address, ~ str_remove(., paste0("[[:punct:][:blank:]]*", as.character(GeoArea), "$")))) %>%

    distinct(Centre, Address, Postcode, GeoArea) %>%
    group_by(Centre) %>%
    add_tally() %>%
    ungroup() %>%
    filter(n > 1) %>%
    select(- n) %>%

    left_join(tmp_df %>%
                  #mutate(across(Address, tools::toTitleCase)) %>%
                  distinct(Centre, CentreName, Address, Postcode, GeoArea)
    ) %>%

    rename_with(~ c("LocalAuthority", "SeedCode", "SchoolName"), c("GeoArea", "Centre", "CentreName")) %>%
    relocate(LocalAuthority) %>%
    relocate(SchoolName, .after = SeedCode) %>%
    anti_join(summary_stats_schools_corrections) %>%

    #pivot_wider(names_from = FlagCorrections, values_from = FlagCorrections) %>%

    group_by(SeedCode) %>%
    add_tally() %>%
    group_by(SeedCode, Address) %>%
    slice_head(n = 1) %>%
    ungroup() %>%
    select(- n) %>%
    
    arrange(SeedCode) %>%
filter(SeedCode %in% c("8110638", "8200831"))
    #write_csv("sta_it_402/data/summary_stats_schools.csv", na = "")

In [ ]:
tmp_df %>%
    filter(Centre %in% c("5240042", "1002848", "5720540", "5720648"))

In [ ]:
tmp_df %>%
    filter(Centre %in% c("8110638", "8200831", "8110131", "5128234"))

In [ ]:

tmp_df %>%
    #distinct(GeoArea, Centre, Address, Postcode) %>%
    distinct(LocalAuthority, SeedCode, Address, Postcode) %>%
    filter(SeedCode %in% c("5509432", "5510430", "5446538", "5446732", "5541638")) %>%

    mutate(across(Address, ~ str_remove(., paste0("[[:punct:][:blank:]]*", as.character(LocalAuthority), "$")), .names = "test"), #GeoArea
           #across(Address, ~ gsub("[[:punct:][:blank:]]*$", "", .)),
          ) %>%
    distinct()

In [ ]:
West Lothian	5509432	Bathgate Academy	Edinburgh Road, Bathgate, West Lothian
West Lothian		Whitburn Academy	Shanks Road, Whitburn
West Lothian	5510430	Whitburn Academy	Shanks Road, Whitburn, West Lothian

Fife		St Columba's R C High School	Woodmill Road, Dunfermline, Fife
Fife	5446538	St Columba's R C High School	Woodmill Road, Dunfermline
Fife		Inverkeithing High School	Hillend Road, Inverkeithing, Fife
Fife	5446732	Inverkeithing High School	Hillend Road, Inverkeithing

In [ ]:
# end test

In [ ]:
*** run utf-8 3008754	Beijing Normal University	

In [ ]:
glimpse(tmp_df)

In [ ]:

#filter(!is.na(FlagCorrections)) %>% 
#filter(Centre %in% c("5110432", "5119936", "5120136", "5123437", "5130239", "5135532", "5142539")) %>%

    distinct() %>% #GeoArea, Centre, CentreName, across(matches("Address|Postcode|Flag"))) %>%



    #pivot_longer(matches("Gender_"), names_prefix = "Gender_", 
    #             names_to = "Gender", values_to = "PupilRoll", values_drop_na = TRUE)


   
#***

#    filter((Country == "Scotland") & if_any(matches("^Correction"), is.na)) %>%
#
#    distinct(GeoArea, Centre, CentreName) %>%
#    left_join(tmp_df) %>%
#
#    distinct(GeoArea, Centre, CentreName, Address, Postcode) %>%
#
#rename_with(~ c("LocalAuthority", "SeedCode", "SchoolName"), c("GeoArea", "Centre", "CentreName")) %>%
#relocate(LocalAuthority) %>%
#anti_join(summary_stats_schools_corrections) %>%
#write_csv("summary_stats_schools.csv", na = "")

In [ ]:
glimpse(centre_postcodes)

centre_postcodes <- centre_postcodes %>%
    anti_join(tmp_df %>%
              mutate(across(everything(), ~ fct_drop(.))) %>%
              rename(Centre = SeedCode)) %>%
    rbind(tmp_df %>%
              mutate(across(everything(), ~ fct_drop(.))) %>%
              rename_with(~ c("Centre", "CentreName", "GeoArea"), c("SeedCode", "SchoolName", "LocalAuthority"))
    ) %>%
    glimpse()

### Derive SIMD and Rural/Urban classifications - Scottish centres

### "external data" - urban/rural classification

In [ ]:
sg_6_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_6_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_6_fold_urban_rural_classification)

sg_8_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_8_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_8_fold_urban_rural_classification)

In [ ]:
sg_6_fold_urban_rural_classification <- sg_6_fold_urban_rural_classification %>%
    rename_with(~ c("URCode", "URClassification", "Description"))  %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural6Fold"))

sg_8_fold_urban_rural_classification <- sg_8_fold_urban_rural_classification %>%
    rename_with(~ c("URCode", "URClassification", "Description"))  %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural8Fold"))

sg_6_fold_urban_rural_classification 
sg_8_fold_urban_rural_classification 


In [ ]:
sg_2_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_2_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_2_fold_urban_rural_classification)

sg_2_fold_urban_rural_classification <- sg_2_fold_urban_rural_classification %>%
    select(- contains("Area")) %>%
    rename_with(~ c("URCode", "URClassification", "Description")) %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural2Fold"))


sg_3_fold_urban_rural_classification <- read_csv("data/sg_urban_rural_classification/sg_3_fold_classification.csv", trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_3_fold_urban_rural_classification)

sg_3_fold_urban_rural_classification <- sg_3_fold_urban_rural_classification %>%
    select(- contains("Area")) %>%
    rename_with(~ c("URCode", "URClassification", "Description")) %>%
    mutate(across(starts_with("UR"), as.factor), 
           across(URClassification, ~ fct_reorder(., as.numeric(URCode)))) %>%
    rename_with(~ str_replace(., "UR", "UrbanRural3Fold"))

sg_2_fold_urban_rural_classification 
sg_3_fold_urban_rural_classification 



In [ ]:
levels(sg_2_fold_urban_rural_classification$UrbanRural2FoldCode)
levels(sg_2_fold_urban_rural_classification$UrbanRural2FoldClassification)

levels(sg_3_fold_urban_rural_classification$UrbanRural3FoldCode)
levels(sg_3_fold_urban_rural_classification$UrbanRural3FoldClassification)

In [ ]:
sg_urban_rural_classification_postcode_lookup <- read_csv("data/sg_urban_rural_classification/sg_2016_urban_rural_classification_postcode_lookup.csv", 
                                                          trim_ws = TRUE, na = c("*", "#", "..")) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                                            filter(rowSums(is.na(.)) != ncol(.)) # caters for skip rows argument not working

glimpse(sg_urban_rural_classification_postcode_lookup)

In [ ]:
# huge file - rather than delete on completion, filter and store subset before join

sg_urban_rural_classification_postcode_lookup <- sg_urban_rural_classification_postcode_lookup %>%

    select(matches("POSTCODE|FOLD|DEL_DATE|LINKPC") | matches("[2368]{1}")) %>%
    rename_with(~ paste0(str_replace(., "UR", "UrbanRural"), "Code"), starts_with("UR")) %>%
    rename(LinkPostcode = LINKPC) %>%
    rename_with(snakecase::to_upper_camel_case) %>%
    mutate(across(c(DelDate, LinkPostcode), ~ na_if(str_trim(.), ""))) %>%
    mutate(across(everything(), as.factor))

glimpse(sg_urban_rural_classification_postcode_lookup)

In [ ]:
sg_urban_rural_classification_postcode_lookup %>%
    select(ends_with("FoldCode")) %>%
    distinct() %>%
    arrange(UrbanRural8FoldCode)

In [ ]:
length(levels(sg_urban_rural_classification_postcode_lookup$Postcode))

sg_urban_rural_classification_postcode_lookup <- sg_urban_rural_classification_postcode_lookup %>%

    filter(is.na(DelDate)) %>%
    filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%

    bind_rows(sg_urban_rural_classification_postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DelDate)) 
    ) %>%
    bind_rows(sg_urban_rural_classification_postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DelDate)) %>%
                distinct(LinkPostcode) %>%
                rename(Postcode = LinkPostcode) %>%

                left_join(sg_urban_rural_classification_postcode_lookup) %>%
                filter(!is.na(Postcode)) 
    ) %>%

    distinct() %>%    
    mutate(across(c(Postcode, DelDate, LinkPostcode), ~ fct_drop(.))) 

length(levels(sg_urban_rural_classification_postcode_lookup$Postcode))

In [ ]:
sg_urban_rural_classification_postcode_lookup %>%
    slice_sample(n = 4)

sg_urban_rural_classification_postcode_lookup %>%
    distinct(across(matches("UrbanRural"))) %>%
    arrange(UrbanRural8FoldCode)

In [ ]:
sg_urban_rural_classification_postcode_lookup %>%
    filter(if_all(matches("UrbanRural"), is.na)) %>%
    distinct() %>%
    arrange(UrbanRural8FoldCode)

In [ ]:
centre_postcodes <- centre_postcodes %>%
    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  filter(is.na(DelDate)) %>%
                  select(- c(DelDate, LinkPostcode))
             ) %>%
    mutate(across(Postcode, as.character), 
           across(Postcode, as.factor)) %>%

    glimpse()


centre_postcodes %>%
    count(Country) %>%
    rename(NoOfCentres = n) %>%
    
    full_join(centre_postcodes %>%
                filter(if_any(starts_with("UrbanRural"), is.na)) %>%
                count(Country) %>%
                rename(URClassificationNotSet = n)
    )

In [ ]:
centre_postcodes %>%
    filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) 
    

In [ ]:
sg_urban_rural_classification_postcode_lookup %>%
    filter(Postcode %in% 
              (centre_postcodes %>%
                filter(if_any(starts_with("UrbanRural"), is.na)) %>%
                distinct(Postcode) %>%
                deframe() %>%
                as.character()  
              ))

In [ ]:
centre_postcodes %>%
    filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
    nrow()

centre_postcodes %>%
    filter((Country == "Scotland") & !(if_any(starts_with("UrbanRural"), is.na))) %>% 
    nrow()

centre_postcodes %>%
    filter((Country == "Scotland") & if_all(starts_with("UrbanRural"), ~ !is.na(.))) %>%
    nrow()


dim(centre_postcodes)

In [ ]:
centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | if_all(starts_with("UrbanRural"), ~ !is.na(.))) %>%

    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
                select(- starts_with("UrbanRural")) %>%

                left_join(centre_postcodes %>%
                            filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
                            distinct(Postcode) %>%

                            left_join(sg_urban_rural_classification_postcode_lookup %>%
                                          filter(!is.na(DelDate)) %>%
                                          select(Postcode, LinkPostcode)) %>%

                            left_join(sg_urban_rural_classification_postcode_lookup %>%
                                          filter(is.na(DelDate)) %>%
                                          select(Postcode, matches("UrbanRural")),
                                      by = c("LinkPostcode" = "Postcode")) 
                          )
                 ) %>%
    glimpse()

length(levels(centre_postcodes$Postcode))


centre_postcodes %>%
    count(Country) %>%
    rename(NoOfCentres = n) %>%
    
    full_join(centre_postcodes %>%
                filter(if_any(starts_with("UrbanRural"), is.na)) %>%
                count(Country) %>%
                rename(URClassificationNotSet = n)
    )

In [ ]:

centre_postcodes %>%
    distinct(across(matches("UrbanRural\\dFoldCode"))) %>%
    arrange(UrbanRural8FoldCode)

In [ ]:
postcode_lookup <- read_csv("data/postcode_lookup/SmallUser.csv", trim_ws = TRUE, na = c("", "NA")) %>% 
                                filter(rowSums(is.na(.)) != ncol(.)) %>%

    bind_rows(read_csv("data/postcode_lookup/LargeUser.csv", trim_ws = TRUE, na = c("", "NA")) %>% 
                                    filter(rowSums(is.na(.)) != ncol(.))) %>%

    glimpse()

In [ ]:
range(postcode_lookup$ScottishIndexOfMultipleDeprivation2020Rank)

In [ ]:
postcode_lookup <- postcode_lookup %>%
    rename(SIMD2020Rank = ScottishIndexOfMultipleDeprivation2020Rank)

simd2020Rank_range <- range(postcode_lookup$SIMD2020Rank)


postcode_lookup <- postcode_lookup %>%
    select(Postcode, DateOfDeletion, LinkedSmallUserPostcode, contains(c("CouncilArea2019", "Settlement2016", "SIMD", "UrbanRural"))) %>%
    distinct() %>%
    mutate(across(where(is.character), ~ str_squish(.)),
           across(c(DateOfDeletion, LinkedSmallUserPostcode), ~ na_if(., "")),
           across(where(is.numeric), as.integer), 
           across(matches("20\\d{2}Code"), as.factor)) %>%
    glimpse()


postcode_lookup %>%
    head()
postcode_lookup %>%
    tail()


In [ ]:
postcode_lookup <- postcode_lookup %>%

    filter(is.na(DateOfDeletion)) %>%
    filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%

    bind_rows(postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DateOfDeletion)) 
    ) %>%
    bind_rows(postcode_lookup %>%
                filter(Postcode %in% levels(centre_postcodes$Postcode)) %>%
                filter(!is.na(DateOfDeletion)) %>%
                distinct(LinkedSmallUserPostcode) %>%
                rename(Postcode = LinkedSmallUserPostcode) %>%

                left_join(postcode_lookup) %>%
                filter(!is.na(Postcode)) 
    ) %>%
        
    distinct() %>%
    glimpse()

In [ ]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    filter(!(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe()))) %>%
    select(EducationAuthority, Centre, CentreName, Postcode, starts_with("UrbanRural")) %>%
    distinct()

In [ ]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    filter(!(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe()))) %>%
    select(EducationAuthority, Centre, CentreName, Postcode, starts_with("UrbanRural")) %>%
    distinct() %>%

    left_join(postcode_lookup %>%
                  filter(LinkedSmallUserPostcode != "NO LINKP") %>%
                  select(Postcode, DateOfDeletion, LinkedSmallUserPostcode) 
    ) %>%

    left_join(postcode_lookup %>%
                  select(Postcode, contains(c("CouncilArea2019", "Settlement2016", "SIMD2020", "UrbanRural"))),
              by = c("LinkedSmallUserPostcode" = "Postcode"),
              na_matches = "never"
    )

In [ ]:
centre_postcodes %>%
    filter((Country == "Scotland") & if_all(starts_with("UrbanRural"), is.na)) %>%
    select(contains(c("Authority", "Centre")), Postcode) %>%
    distinct() %>%
    arrange(EducationAuthority)

In [ ]:
# fill in remaining blanks... 

centre_postcodes %>%
    filter((Country == "Scotland") & if_any(starts_with("UrbanRural"), is.na)) %>%
    select(- starts_with("UrbanRural")) %>%
    
    filter(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe())) %>%

    left_join(postcode_lookup %>%
                  select(Postcode, contains(c("CouncilArea2019", "Settlement2016", "IndexOfMultipleDeprivation", "UrbanRural")))
             ) %>%
    rename_with(~ str_replace(., "2016", ""), starts_with("UrbanRural")) %>%
    

    left_join(sg_6_fold_urban_rural_classification %>%
                  select(- Description)) %>%
    left_join(sg_8_fold_urban_rural_classification %>%
                 select(- Description)) %>%

    select(EducationAuthority, Centre, CentreName, Postcode, contains(c("CouncilArea2019", "Settlement2016", "IndexOfMultipleDeprivation", "UrbanRural"))) 


In [ ]:
postcode_lookup %>%
    anti_join(sg_urban_rural_classification_postcode_lookup) %>%
    rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  distinct(across(matches("^UrbanRural\\d"))) %>%
                  drop_na()
             ) %>%
    distinct(across(matches("^(UrbanRural\\dFold)"))) %>%
    relocate(UrbanRural2FoldCode, UrbanRural3FoldCode)

sg_urban_rural_classification_postcode_lookup %>%
    distinct(across(matches("^(UrbanRural\\dFold)")))


In [ ]:
postcode_lookup %>%
    anti_join(sg_urban_rural_classification_postcode_lookup) %>%
    rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  distinct(across(matches("^UrbanRural\\d"))) %>%
                  drop_na()
             ) %>%
    relocate(UrbanRural2FoldCode, UrbanRural3FoldCode, .before = UrbanRural6FoldCode)

In [ ]:
postcode_lookup %>%
    filter(Postcode == "G2 3BZ") 

In [ ]:
tmp_df <- centre_postcodes %>%
    filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na))

tmp_df <- tmp_df %>%
    select(- starts_with("UrbanRural")) %>%

    filter(Postcode %in% 
              (postcode_lookup %>%
                   filter(is.na(DateOfDeletion)) %>%
                   distinct(Postcode) %>%
                   deframe())) %>%

    left_join(postcode_lookup %>%
                  filter(is.na(DateOfDeletion)) %>%
                  select(Postcode, LinkedSmallUserPostcode, matches("^UrbanRural\\d"))
             ) %>%
    rename_with(~ str_replace(., "2016", ""), matches("^UrbanRural\\d")) %>%

    rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

    left_join(sg_urban_rural_classification_postcode_lookup %>%
                  distinct(across(matches("^UrbanRural\\d"))) %>%
                  drop_na()
             ) 

tmp_df %>%
    select(matches("^(Centre|Postcode|UrbanRural)"))

centre_postcodes %>%
    filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na)) %>%

    anti_join(tmp_df, by = "Centre"
             ) %>%
    select(matches("^(Centre|Postcode|UrbanRural)"))

centre_postcodes %>%
    filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na))  %>%
    select(- matches("UrbanRural|DateOfDeletion")) %>%

    inner_join(postcode_lookup %>%
                   filter(!is.na(DateOfDeletion)) %>%
                   distinct()
             ) 

In [ ]:
tmp_df %>%
    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na)) %>%
                select(- starts_with("UrbanRural")) %>%

                anti_join(tmp_df, by = "Centre"
                         ) %>%
                
                left_join(postcode_lookup %>%
                              filter(!is.na(DateOfDeletion)) %>%
                              select(Postcode, LinkedSmallUserPostcode)) %>%

                left_join(postcode_lookup %>%
                              filter(is.na(DateOfDeletion)) %>%
                              select(Postcode, matches("UrbanRural")),
                          by = c("LinkedSmallUserPostcode" = "Postcode")
                         ) %>% 
              
                rename_with(~ str_replace(., "2016", ""), matches("^UrbanRural\\d")) %>%

                rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

                left_join(sg_urban_rural_classification_postcode_lookup %>%
                              distinct(across(matches("^UrbanRural\\d"))) %>%
                              drop_na()
                         ) 
                ) 

In [ ]:
# fill in remaining blanks... 

centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | if_all(starts_with("UrbanRural"), ~ !is.na(.))) %>%

    rbind(tmp_df %>%
                bind_rows(centre_postcodes %>%
                            filter((Country == "Scotland") & if_any(matches("^UrbanRural\\d"), is.na)) %>%
                            select(- starts_with("UrbanRural")) %>%

                            anti_join(tmp_df, by = "Centre"
                                     ) %>%

                            left_join(postcode_lookup %>%
                                          filter(!is.na(DateOfDeletion)) %>%
                                          select(Postcode, LinkedSmallUserPostcode)) %>%

                            left_join(postcode_lookup %>%
                                          filter(is.na(DateOfDeletion)) %>%
                                          select(Postcode, matches("UrbanRural")),
                                      by = c("LinkedSmallUserPostcode" = "Postcode")
                                     ) %>% 

                            rename_with(~ str_replace(., "2016", ""), matches("^UrbanRural\\d")) %>%

                            rename_with(~ gsub("^(UrbanRural\\dFold)(\\d+)(C\\w+)", "\\1\\3", ., ignore.case = TRUE)) %>%

                            left_join(sg_urban_rural_classification_postcode_lookup %>%
                                          distinct(across(matches("^UrbanRural\\d"))) %>%
                                          drop_na()
                                     ) 
                            ) %>%
                select(names(centre_postcodes)) 
              
             ) %>%

    glimpse()

#### END &ndash; "external data" &ndash; urban/rural classification

### SIMD
* using the Scottish Index of Multiple Deprivation 2020 rankings

In [ ]:
postcode_lookup %>%
    select(Postcode, SIMD2020Rank, DateOfDeletion, LinkedSmallUserPostcode) %>%
    group_by(Postcode, SIMD2020Rank) %>%
    add_tally() %>%
    filter(n > 1) %>%
    #arrange(Postcode, desc(DateOfDeletion), SIMD2020Rank)
    distinct(Postcode) %>%
    nrow()

postcode_lookup %>%
    select(Postcode, SIMD2020Rank, DateOfDeletion, LinkedSmallUserPostcode) %>%
    group_by(Postcode, SIMD2020Rank, DateOfDeletion) %>%
    add_tally() %>%
    filter(n > 1)

In [ ]:
centre_postcodes <- centre_postcodes %>%
    left_join(postcode_lookup %>%
                  filter(is.na(DateOfDeletion)) %>%
                  select(Postcode, SIMD2020Rank) 
             )

centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
    distinct(EducationAuthority, Centre, CentreName, Postcode)

In [ ]:
tmp_df <- centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
    select(- SIMD2020Rank) %>%

    left_join(postcode_lookup %>%
                  filter(!is.na(DateOfDeletion) & (LinkedSmallUserPostcode != "NO LINKP")) %>%
                  distinct(Postcode, LinkedSmallUserPostcode) 
             ) %>%

    left_join(postcode_lookup %>%
                  distinct(Postcode, SIMD2020Rank),
              by = c("LinkedSmallUserPostcode" = "Postcode")
    )

# remaining matches not in use anywhere else; use (deleted ranking)?
#tmp_df%>%
#    filter(!is.na(SIMD2020Rank)) %>%
#
#    bind_rows(tmp_df %>%
#
#                filter(is.na(SIMD2020Rank)) %>%
#                select(- SIMD2020Rank) %>%
#
#                left_join(postcode_lookup %>%
#                              select(Postcode, SIMD2020Rank) 
#                         )
#              )

In [ ]:
glimpse(centre_postcodes)

In [ ]:
centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | !is.na(SIMD2020Rank)) %>%
 
    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
                select(- SIMD2020Rank) %>%

                left_join(postcode_lookup %>%
                              filter(!is.na(DateOfDeletion) & (LinkedSmallUserPostcode != "NO LINKP")) %>%
                              distinct(Postcode, LinkedSmallUserPostcode) 
                         ) %>%

                left_join(postcode_lookup %>%
                              distinct(Postcode, SIMD2020Rank),
                          by = c("LinkedSmallUserPostcode" = "Postcode")
                )
             ) %>%
    relocate(SIMD2020Rank, .after = last_col()) %>%
    glimpse()


centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank))

In [ ]:
# checking remainder ...

centre_postcodes %>%
    filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
    distinct(Postcode, across(matches("Centre|UrbanRural8FoldCode|SIMD2020|Link"))) %>%

    select(- SIMD2020Rank) %>%
    left_join(postcode_lookup %>%
                  distinct(Postcode, DateOfDeletion, SIMD2020Rank) 
             ) 

# ranks not reused elsewhere - include matches and flag deletion

In [ ]:
centre_postcodes <- centre_postcodes %>%
    filter((Country != "Scotland") | !is.na(SIMD2020Rank)) %>%
 
    bind_rows(centre_postcodes %>%
                filter((Country == "Scotland") & is.na(SIMD2020Rank)) %>%
                select(- SIMD2020Rank) %>%

                left_join(postcode_lookup %>%
                              distinct(Postcode, DateOfDeletion, SIMD2020Rank) 
                         ) 
             ) %>%
    relocate(SIMD2020Rank, .after = last_col()) %>%
    mutate(across(DateOfDeletion, ~ if_else(!is.na(.), paste0("Postcode metadata ", cur_column(), ": ", .), .))) %>%
    rename(Flag = DateOfDeletion) %>%
    glimpse()


In [ ]:
centre_postcodes %>%
    filter(Country == "Scotland") %>%
    
    left_join(centre_postcodes %>%
                  filter(Country == "Scotland") %>%
                  arrange(SIMD2020Rank) %>%
                  mutate(SIMD2020RelativeRank = rank(SIMD2020Rank, ties.method = "min", na.last = "keep")) 
         ) %>%
    #relocate(SIMD2020Rank, .before = SIMD2020RelativeRank) 
    distinct(Postcode, across(matches("Centre|SIMD2020"))) %>%
    arrange(SIMD2020Rank)


In [ ]:
dim(postcode_lookup)

simd2020Rank_range
quantile(as.integer(range(simd2020Rank_range)), probs = seq(0, 1, 1/5))

In [ ]:
simd2020_quintiles <- enframe(quantile(as.integer(simd2020Rank_range), probs = seq(0, 1, 1/5))) %>%
    rename_with(~ c("quintile", "rank")) %>%
    mutate(across(quintile, ~ str_remove(., "%")), 
           across(quintile, as.integer), 
           across(quintile, ~ (. / 20)), 
           across(everything(), as.integer)
          ) %>%
    filter(quintile > 0)

simd2020_quintiles

In [ ]:
getSimdQuintile <-
    function(value, quintiles) {
        
        if (is.na(value))
            return(NA)
        
        row_number <- min(which(value <= deframe(quintiles["rank"])))
        return(deframe(quintiles["quintile"])[[row_number]])
    }

#getSimdQuintile <- Vectorize(getSimdQuintile)

In [ ]:
getSimdQuintile(6976, simd2020_quintiles)
getSimdQuintile(NA, simd2020_quintiles)

sapply(c(1743, 3336, 5367, 2530, 6146, 0027, 0023, 6921, 4192), getSimdQuintile, simd2020_quintiles)

In [ ]:
centre_postcodes$SIMD2020Quintile <-
    sapply(centre_postcodes$SIMD2020Rank, 
           getSimdQuintile, 
           simd2020_quintiles)

head(centre_postcodes$SIMD2020Quintile, 15)

In [ ]:
centre_postcodes <- centre_postcodes %>%
    
    left_join(centre_postcodes %>%
                  filter(Country == "Scotland") %>%
                  arrange(SIMD2020Rank) %>%
                  mutate(SIMD2020RelativeRank = rank(SIMD2020Rank, ties.method = "min", na.last = "keep")) 
         ) %>%
    relocate(SIMD2020Rank, SIMD2020RelativeRank, SIMD2020Quintile, .before = Flag) %>%
    glimpse()


centre_postcodes %>%
    group_by(SIMD2020Quintile) %>%
    slice_sample(n = 2)

#### END &ndash; SIMD Ranking

### Write to DataStore

In [ ]:
dbGetQuery(dbConn, "SELECT COUNT(*) FROM centre_postcodes")
dbGetQuery(dbConn, "SELECT * FROM centre_postcodes LIMIT 6")

centre_postcodes %>%
    select(- matches("Link")) %>%

    left_join(sg_2_fold_urban_rural_classification %>%
                              select(- Description)) %>%
    left_join(sg_3_fold_urban_rural_classification %>%
                              select(- Description)) %>%
    left_join(sg_6_fold_urban_rural_classification %>%
                              select(- Description)) %>%
    left_join(sg_8_fold_urban_rural_classification %>%
                              select(- Description)) %>%
                              
    relocate(UrbanRural2FoldCode, UrbanRural2FoldClassification,
             UrbanRural3FoldCode, UrbanRural3FoldClassification,
             UrbanRural6FoldCode, UrbanRural6FoldClassification,
             UrbanRural8FoldCode, UrbanRural8FoldClassification, .before = SIMD2020Rank) %>%
    glimpse()

In [ ]:
writeToDataStore(centre_postcodes %>%
                    select(- matches("Link")) %>%

                    left_join(sg_2_fold_urban_rural_classification %>%
                                              select(- Description)) %>%
                    left_join(sg_3_fold_urban_rural_classification %>%
                                              select(- Description)) %>%
                    left_join(sg_6_fold_urban_rural_classification %>%
                                              select(- Description)) %>%
                    left_join(sg_8_fold_urban_rural_classification %>%
                                              select(- Description)) %>%

                    relocate(UrbanRural2FoldCode, UrbanRural2FoldClassification,
                             UrbanRural3FoldCode, UrbanRural3FoldClassification,
                             UrbanRural6FoldCode, UrbanRural6FoldClassification,
                             UrbanRural8FoldCode, UrbanRural8FoldClassification, .before = SIMD2020Rank), 
                 dbConn, "centre_postcodes", TRUE)
                 
dbGetQuery(dbConn, "SELECT COUNT(*) FROM centre_postcodes")
dbGetQuery(dbConn, "SELECT DISTINCT GeoArea FROM centre_postcodes")

In [ ]:
dbDisconnect(dbConn)